<a href="https://colab.research.google.com/github/racoope70/exploratory-daytrading/blob/main/ppo_alpaca_paper_trading_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#Clean any partials
!pip uninstall -y stable-baselines3 shimmy gymnasium gym autorom AutoROM.accept-rom-license ale-py

#Install the compatible trio (no [extra] to avoid Atari deps)
!pip install "gymnasium==0.29.1" "shimmy==1.3.0" "stable-baselines3==2.3.0"

#Your other libs (safe to keep separate)
!pip install alpaca-trade-api ta python-dotenv gym-anytrading


Found existing installation: gymnasium 1.2.3
Uninstalling gymnasium-1.2.3:
  Successfully uninstalled gymnasium-1.2.3
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
Found existing installation: ale-py 0.11.2
Uninstalling ale-py-0.11.2:
  Successfully uninstalled ale-py-0.11.2
Reason for being yanked: Loading broken with PyTorch 1.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires ale-py>=0.10.1, which is not installed.
dopamine-rl 4.1.2 requires gym<=0.25.2, which is not installed.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.1 which is incompatible.
  Preparing metadata (setup.py) ... done
   

In [ ]:
import torch, gymnasium, shimmy, stable_baselines3 as sb3
import alpaca_trade_api, websockets, pywt

print("torch:", torch.__version__)
print("gymnasium:", gymnasium.__version__)
print("shimmy:", shimmy.__version__)
print("stable-baselines3:", sb3.__version__)
print("alpaca-trade-api:", alpaca_trade_api.__version__)
print("websockets:", websockets.__version__)
print("pywavelets:", pywt.__version__)


torch: 2.9.0+cpu
gymnasium: 0.29.1
shimmy: 1.3.0
stable-baselines3: 2.3.0
alpaca-trade-api: 3.2.0
websockets: 10.4
pywavelets: 1.8.0


In [ ]:
# Colab: install into the current kernel
!pip -q install alpaca-trade-api==3.2.0 python-dotenv ta gym-anytrading

In [ ]:
import alpaca_trade_api
print("alpaca-trade-api:", alpaca_trade_api.__version__)

alpaca-trade-api: 3.2.0


In [ ]:
from __future__ import annotations
import csv
import sys
import gc
import json
import logging
import math
import os
import pickle
import re
import shutil
import time
import warnings
from datetime import datetime, timedelta, timezone
from decimal import Decimal, ROUND_DOWN, ROUND_HALF_UP
from functools import lru_cache
from pathlib import Path
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union

# Scientific / data stack
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from dataclasses import dataclass, field

# Fix seed for reusability
import random
import torch

os.environ["PYTHONHASHSEED"] = "0"
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# Alpaca trading API
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame

# Try to use TimeFrameUnit if your alpaca_trade_api version has it
try:
    from alpaca_trade_api.rest import TimeFrameUnit  # type: ignore
    _TF_MAP = {
        "1min": TimeFrame.Minute,
        "1minute": TimeFrame.Minute,
        "5min": TimeFrame(5, TimeFrameUnit.Minute),
        "15min": TimeFrame(15, TimeFrameUnit.Minute),
        "1h": TimeFrame.Hour,
        "1hour": TimeFrame.Hour,
        "60min": TimeFrame.Hour,
        "1d": TimeFrame.Day,
        "day": TimeFrame.Day,
    }
except Exception:
    # alpaca_trade_api supports timeframe strings like "5Min", "15Min", etc.
    _TF_MAP = {
        "1min": "1Min",
        "1minute": "1Min",
        "5min": "5Min",
        "15min": "15Min",
        "1h": "1Hour",     # (if your env doesn't accept this, use "60Min")
        "1hour": "1Hour",
        "60min": "60Min",
        "1d": "1Day",
        "day": "1Day",
    }

LIVE_TIMEFRAME = "1Hour"  # placeholder; overwritten in __main__

# RL models
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

# (Optional) Colab helpers
IN_COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import drive, files  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# ============================================================
# Utils / Paths / Global cooldowns
# ============================================================

def round_to_cents(x: float) -> float:
    return float(Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_DOWN))

if IN_COLAB:
    try:
        drive.mount("/content/drive", force_remount=False)
    except Exception:
        pass

# Project root (Drive in Colab; cwd locally)
if IN_COLAB:
    PROJECT_ROOT = Path("/content/drive/MyDrive/AlpacaPaper")
else:
    PROJECT_ROOT = Path.cwd() / "AlpacaPaper"
PROJECT_ROOT.mkdir(parents=True, exist_ok=True)

# Order throttling timestamps (per symbol)
_ORDER_EVENT_TS: Dict[str, float] = {}
_LAST_ORDER_TS: Dict[str, float] = {}

# One-time seed-buy flags (per symbol) for FORCE_FIRST_BUY
_FORCED_FIRST_BUY_DONE: Dict[str, bool] = {}

def begin_order_event(symbol: str, min_gap_sec: int) -> bool:
    now = time.time()
    last = _ORDER_EVENT_TS.get(symbol, 0.0)
    if (now - last) < float(min_gap_sec):
        return False
    _ORDER_EVENT_TS[symbol] = now
    return True

def stamp_order_event(symbol: str) -> None:
    ts = time.time()
    _ORDER_EVENT_TS[symbol] = ts
    _LAST_ORDER_TS[symbol] = ts

SESSION_OPEN_EQUITY: Optional[float] = None
_last_kill_ts: float = 0.0

# Faster cooldown for the very first seed fill
_SEED_COOLDOWN_SEC = 10

_NO_POS_CYCLE_COUNT: Dict[str, int] = {}

# Re-entry cooldown after forced flatten
_REENTRY_BLOCK_UNTIL: Dict[str, float] = {}
REENTRY_COOLDOWN_SEC = int(os.getenv("REENTRY_COOLDOWN_SEC", "300"))

def _too_soon(symbol: str, min_gap_sec: int = 30) -> bool:
    now = time.time()
    last = _LAST_ORDER_TS.get(symbol, 0.0)
    if (now - last) < float(min_gap_sec):
        return True
    _LAST_ORDER_TS[symbol] = now
    return False

def _to_bool(x: str) -> bool:
    return str(x).strip().lower() in ("1", "true", "yes", "y", "on")

def env_bool(name: str, default: str = "0") -> bool:
    return _to_bool(os.getenv(name, default))

# ============================================================
# Upload / Conversion Helpers (Colab)
# ============================================================

def can_short_symbol(api, symbol: str) -> bool:
    if not bool(globals().get("ALLOW_SHORTS", False)):
        return False

    try:
        acct = api.get_account()
        shorting_ok = bool(getattr(acct, "shorting_enabled", False))
    except Exception:
        shorting_ok = False

    try:
        asset = api.get_asset(symbol)
        asset_ok = bool(getattr(asset, "shortable", False))
    except Exception:
        asset_ok = False

    return shorting_ok and asset_ok

def upload_env_and_artifacts_in_colab():
    if not IN_COLAB:
        return

    target_dir = Path(os.getenv("ARTIFACTS_DIR", str(PROJECT_ROOT / "artifacts")))
    target_dir.mkdir(parents=True, exist_ok=True)

    print("Upload your .env (or Alpaca_keys.env.txt). Cancel if already on Drive.")
    up = files.upload()
    if up:
        if "Alpaca_keys.env.txt" in up:
            src = Path("Alpaca_keys.env.txt")
            dst = PROJECT_ROOT / ".env"
            shutil.move(str(src), str(dst))
            print(f"Saved env → {dst}")
        elif ".env" in up:
            src = Path(".env")
            dst = PROJECT_ROOT / ".env"
            shutil.move(str(src), str(dst))
            print(f"Saved env → {dst}")
        else:
            any_name = next(iter(up.keys()))
            src = Path(any_name)
            dst = PROJECT_ROOT / ".env"
            shutil.move(str(src), str(dst))
            print(f"Saved env (renamed {any_name}) → {dst}")

    print("Upload your artifacts (ppo_*_model.zip, *_vecnorm*.pkl, *_features*.json or .txt).")
    up2 = files.upload()
    for name in up2.keys():
        shutil.move(name, target_dir / name)
    print("Artifacts now in:", sorted(p.name for p in target_dir.iterdir()))

def _maybe_convert_features_txt_to_json():
    """Convert any 'features_<TICKER>.txt' into 'ppo_<TICKER>_features.json' (simple list)."""
    art_dir = Path(os.getenv("ARTIFACTS_DIR", str(PROJECT_ROOT / "artifacts")))
    art_dir.mkdir(parents=True, exist_ok=True)
    for p in art_dir.glob("features_*.txt"):
        ticker = re.sub(r"^features_|\.txt$", "", p.name, flags=re.IGNORECASE)
        try:
            raw = p.read_text().strip()
            items = [x.strip() for x in raw.replace(",", "\n").splitlines() if x.strip()]
            out = {"features": items}
            out_path = art_dir / f"ppo_{ticker}_features.json"
            out_path.write_text(json.dumps(out, indent=2))
            print(f"Converted {p.name} → {out_path.name}  ({len(items)} features)")
        except Exception as e:
            print(f"Could not convert {p.name}: {e}")

def _maybe_rename_vecnorm_scaler():
    """Rename any 'scaler_<TICKER>.pkl' to 'ppo_<TICKER>_vecnorm.pkl'."""
    art_dir = Path(os.getenv("ARTIFACTS_DIR", str(PROJECT_ROOT / "artifacts")))
    art_dir.mkdir(parents=True, exist_ok=True)
    for p in art_dir.glob("scaler_*.pkl"):
        ticker = re.sub(r"^scaler_|\.pkl$", "", p.name, flags=re.IGNORECASE)
        dst = art_dir / f"ppo_{ticker}_vecnorm.pkl"
        if not dst.exists():
            shutil.move(str(p), str(dst))
            print(f"Renamed {p.name} → {dst.name}")

def normalize_artifacts():
    _maybe_convert_features_txt_to_json()
    _maybe_rename_vecnorm_scaler()

# ============================================================
# Env & logging (initial) + .env loading
# ============================================================

warnings.filterwarnings("default")

# Load env (supports PROJECT_ROOT/.env)
env_candidates = [PROJECT_ROOT / ".env", Path(".env")]
for env_path in env_candidates:
    if env_path.exists():
        load_dotenv(dotenv_path=env_path, override=True)
        break
else:
    load_dotenv(override=True)

# ---- Re-read env booleans AFTER .env is loaded ----
# Keep your existing reentry env read
REENTRY_COOLDOWN_SEC = int(os.getenv("REENTRY_COOLDOWN_SEC", "300"))

# Default timeouts / reporting cadence
os.environ.setdefault("PH_TIMEOUT_SEC", "8")
os.environ.setdefault("EQUITY_TIMEFRAME", "5Min")

# Debug idle-seed knobs
os.environ.setdefault("DEBUG_FORCE_SEED_IF_IDLE", "0")
os.environ.setdefault("DEBUG_SEED_IDLE_CYCLES", "10")

# Basic console logger (replaced with file+console after RESULTS_DIR exists)
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")
logging.getLogger().setLevel(getattr(logging, os.getenv("LOG_LEVEL", "INFO").upper(), logging.INFO))

root = logging.getLogger()
root.handlers.clear()

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(getattr(logging, os.getenv("LOG_LEVEL", "INFO").upper(), logging.INFO))
handler.setFormatter(logging.Formatter("%(asctime)s | %(levelname)s | %(message)s"))
root.addHandler(handler)
root.setLevel(handler.level)

try:
    sys.stdout.reconfigure(line_buffering=True)
except Exception:
    pass

# ============================================================
# Config dataclass
# ============================================================


def _to_list_csv(x: str) -> list:
    return [s.strip().upper() for s in str(x).split(",") if s.strip()]

@dataclass
class Knobs:
    # API / mode
    APCA_API_BASE_URL: str = "https://paper-api.alpaca.markets"
    DRY_RUN: bool = False
    AUTO_RUN_LIVE: bool = True
    INF_DETERMINISTIC: bool = True

    FLATTEN_INTO_CLOSE: bool = False
    FORCE_FIRST_BUY: bool = False
    FORCE_FLATTEN_ON_EXIT: bool = False

    # Equity logging controls
    EQUITY_LOG_THROTTLE_SEC: int = 900
    SKIP_EQUITY_WHEN_DRY_RUN: bool = True

    # Universe / files
    TICKERS: List[str] = field(default_factory=list)
    ARTIFACTS_DIR: str = ""
    RESULTS_ROOT: str = ""

    # Data feed / cadence / staleness
    BARS_FEED: str = "iex"
    COOLDOWN_MIN: int = 10
    STALE_MAX_SEC: int = 4200

    # Sizing & entry/exit sensitivity
    SIZING_MODE: str = "threshold"  # "linear" | "threshold"
    WEIGHT_CAP: float = 0.35
    CONF_FLOOR: float = 0.15
    ENTER_CONF_MIN: float = 0.12
    ENTER_WEIGHT_MIN: float = 0.02
    EXIT_WEIGHT_MAX: float = 0.008
    REBALANCE_MIN_NOTIONAL: float = 10.00
    USE_FRACTIONALS: bool = True
    SEED_FIRST_SHARE: bool = True
    ALLOW_SHORTS: bool = False

    # add-ons
    DELTA_WEIGHT_MIN: float = 0.002
    RAW_POS_MIN: float = 0.00
    RAW_NEG_MAX: float = 0.00

    # Risk
    TAKE_PROFIT_PCT: float = 0.05
    STOP_LOSS_PCT: float = 0.03

    # Timeframe intent
    TRAIN_TIMEFRAME: str = "1H"
    DATA_TIMEFRAME: str = "1H"
    EQUITY_TIMEFRAME: str = "5Min"

    # Kill-switch
    MAX_DAILY_DRAWDOWN_PCT: float = 0.05
    KILL_SWITCH_COOLDOWN_MIN: int = 30

    # Exit policy
    EXIT_AFTER_CLOSE: bool = False

    # Secrets
    APCA_API_KEY_ID: str = ""
    APCA_API_SECRET_KEY: str = ""

    # Misc
    STALE_BEST_WINDOW: str = ""

    @classmethod
    def from_env(
        cls,
        defaults: "Knobs",
        project_root: Path,
        env: Mapping[str, str],
        overrides: Mapping[str, object] = None,
    ):
        kv = {**defaults.__dict__}
        kv.update({
          "APCA_API_BASE_URL": env.get("APCA_API_BASE_URL", kv["APCA_API_BASE_URL"]),
          "AUTO_RUN_LIVE":     _to_bool(env.get("AUTO_RUN_LIVE", str(kv["AUTO_RUN_LIVE"]))),
          "DRY_RUN":           _to_bool(env.get("DRY_RUN", str(kv["DRY_RUN"]))),
          "INF_DETERMINISTIC": _to_bool(env.get("INF_DETERMINISTIC", str(kv["INF_DETERMINISTIC"]))),

          # --- execution policy flags (only once) ---
          "FLATTEN_INTO_CLOSE": _to_bool(env.get("FLATTEN_INTO_CLOSE", str(kv.get("FLATTEN_INTO_CLOSE", False)))),
          "FORCE_FIRST_BUY": _to_bool(env.get("FORCE_FIRST_BUY", str(kv.get("FORCE_FIRST_BUY", False)))),
          "FORCE_FLATTEN_ON_EXIT": _to_bool(env.get("FORCE_FLATTEN_ON_EXIT", str(kv.get("FORCE_FLATTEN_ON_EXIT", False)))),

          "EQUITY_LOG_THROTTLE_SEC": int(env.get("EQUITY_LOG_THROTTLE_SEC", str(kv["EQUITY_LOG_THROTTLE_SEC"]))),
          "SKIP_EQUITY_WHEN_DRY_RUN": _to_bool(env.get("SKIP_EQUITY_WHEN_DRY_RUN", str(kv["SKIP_EQUITY_WHEN_DRY_RUN"]))),

          "USE_FRACTIONALS": _to_bool(env.get("USE_FRACTIONALS", str(kv["USE_FRACTIONALS"]))),
          "SEED_FIRST_SHARE": _to_bool(env.get("SEED_FIRST_SHARE", str(kv["SEED_FIRST_SHARE"]))),
          "ALLOW_SHORTS": _to_bool(env.get("ALLOW_SHORTS", str(kv["ALLOW_SHORTS"]))),

          "TICKERS": _to_list_csv(env.get("TICKERS", ",".join(kv["TICKERS"] or ["UNH", "GE"]))),
          "ARTIFACTS_DIR": env.get("ARTIFACTS_DIR", kv["ARTIFACTS_DIR"] or str(project_root / "artifacts")),
          "RESULTS_ROOT": env.get("RESULTS_ROOT", kv["RESULTS_ROOT"] or str(project_root / "results")),

          "BARS_FEED": env.get("BARS_FEED", kv["BARS_FEED"]),
          "COOLDOWN_MIN": int(env.get("COOLDOWN_MIN", str(kv["COOLDOWN_MIN"])) or kv["COOLDOWN_MIN"]),
          "STALE_MAX_SEC": int(env.get("STALE_MAX_SEC", str(kv["STALE_MAX_SEC"])) or kv["STALE_MAX_SEC"]),

          "SIZING_MODE": env.get("SIZING_MODE", kv["SIZING_MODE"]),
          "WEIGHT_CAP": float(env.get("WEIGHT_CAP", str(kv["WEIGHT_CAP"]))),
          "CONF_FLOOR": float(env.get("CONF_FLOOR", str(kv["CONF_FLOOR"]))),
          "ENTER_CONF_MIN": float(env.get("ENTER_CONF_MIN", str(kv["ENTER_CONF_MIN"]))),
          "ENTER_WEIGHT_MIN": float(env.get("ENTER_WEIGHT_MIN", str(kv["ENTER_WEIGHT_MIN"]))),
          "EXIT_WEIGHT_MAX": float(env.get("EXIT_WEIGHT_MAX", str(kv["EXIT_WEIGHT_MAX"]))),
          "REBALANCE_MIN_NOTIONAL": float(env.get("REBALANCE_MIN_NOTIONAL", str(kv["REBALANCE_MIN_NOTIONAL"]))),

          "TAKE_PROFIT_PCT": float(env.get("TAKE_PROFIT_PCT", str(kv["TAKE_PROFIT_PCT"]))),
          "STOP_LOSS_PCT": float(env.get("STOP_LOSS_PCT", str(kv["STOP_LOSS_PCT"]))),

          "DELTA_WEIGHT_MIN": float(env.get("DELTA_WEIGHT_MIN", str(kv.get("DELTA_WEIGHT_MIN", 0.002)))),
          "RAW_POS_MIN": float(env.get("RAW_POS_MIN", str(kv.get("RAW_POS_MIN", 0.0)))),
          "RAW_NEG_MAX": float(env.get("RAW_NEG_MAX", str(kv.get("RAW_NEG_MAX", 0.0)))),

          "EXIT_AFTER_CLOSE": _to_bool(env.get("EXIT_AFTER_CLOSE", str(kv.get("EXIT_AFTER_CLOSE", False)))),

          "STALE_BEST_WINDOW": env.get("STALE_BEST_WINDOW", kv.get("STALE_BEST_WINDOW", "")),
          "DATA_TIMEFRAME": env.get("DATA_TIMEFRAME", kv.get("DATA_TIMEFRAME", "1H")),
          "TRAIN_TIMEFRAME": env.get("TRAIN_TIMEFRAME", kv.get("TRAIN_TIMEFRAME", "1H")),
          "EQUITY_TIMEFRAME": env.get("EQUITY_TIMEFRAME", kv.get("EQUITY_TIMEFRAME", "5Min")),
      })

        kv["APCA_API_KEY_ID"] = env.get("APCA_API_KEY_ID") or env.get("ALPACA_API_KEY_ID", "") or ""
        kv["APCA_API_SECRET_KEY"] = env.get("APCA_API_SECRET_KEY") or env.get("ALPACA_API_SECRET_KEY", "") or ""

        if overrides:
            for k, v in overrides.items():
                key = str(k)
                if key.upper() == "TICKERS" and isinstance(v, str):
                    v = _to_list_csv(v)
                kv[key] = v

        return cls(**kv)

    def apply_to_globals(self):
        g = globals()
        g["BASE_URL"] = self.APCA_API_BASE_URL
        g["DRY_RUN"] = bool(self.DRY_RUN)
        g["INF_DETERMINISTIC"] = bool(self.INF_DETERMINISTIC)
        g["TICKERS"] = list(self.TICKERS or ["UNH", "GE"])

        # Paths
        g["ARTIFACTS_DIR"] = Path(self.ARTIFACTS_DIR)
        g["RESULTS_ROOT"] = Path(self.RESULTS_ROOT)
        g["RESULTS_DIR"] = g["RESULTS_ROOT"] / datetime.now(timezone.utc).strftime("%Y-%m-%d")
        g["LATEST_DIR"] = g["RESULTS_ROOT"] / "latest"

        for p in (g["ARTIFACTS_DIR"], g["RESULTS_DIR"], g["LATEST_DIR"]):
            p.mkdir(parents=True, exist_ok=True)

        results_dir = g["RESULTS_DIR"]
        latest_dir = g["LATEST_DIR"]

        g["BARS_FEED"] = str(self.BARS_FEED).strip()
        g["COOLDOWN_MIN"] = int(self.COOLDOWN_MIN)
        g["STALE_MAX_SEC"] = int(self.STALE_MAX_SEC)
        g["SIZING_MODE"] = self.SIZING_MODE
        g["WEIGHT_CAP"] = float(self.WEIGHT_CAP)
        g["ENTER_CONF_MIN"] = float(self.ENTER_CONF_MIN)
        g["ENTER_WEIGHT_MIN"] = float(self.ENTER_WEIGHT_MIN)
        g["EXIT_WEIGHT_MAX"] = float(self.EXIT_WEIGHT_MAX)
        g["REBALANCE_MIN_NOTIONAL"] = float(self.REBALANCE_MIN_NOTIONAL)
        g["USE_FRACTIONALS"] = bool(self.USE_FRACTIONALS)
        g["SEED_FIRST_SHARE"] = bool(self.SEED_FIRST_SHARE)
        g["ALLOW_SHORTS"] = bool(self.ALLOW_SHORTS)
        g["CONF_FLOOR"] = float(self.CONF_FLOOR)
        g["TAKE_PROFIT_PCT"] = float(self.TAKE_PROFIT_PCT)
        g["STOP_LOSS_PCT"] = float(self.STOP_LOSS_PCT)
        g["BEST_WINDOW_ENV"] = (self.STALE_BEST_WINDOW or None)
        g["API_KEY"] = self.APCA_API_KEY_ID or ""
        g["API_SECRET"] = self.APCA_API_SECRET_KEY or ""

        g["DELTA_WEIGHT_MIN"] = float(self.DELTA_WEIGHT_MIN)
        g["RAW_POS_MIN"] = float(self.RAW_POS_MIN)
        g["RAW_NEG_MAX"] = float(self.RAW_NEG_MAX)

        g["TRADE_LOG_CSV"] = results_dir / "trade_log_master.csv"
        g["EQUITY_LOG_CSV"] = results_dir / "equity_log.csv"
        g["PLOT_PATH"] = results_dir / "equity_curve.png"
        g["PLOT_PATH_LATEST"] = latest_dir / "equity_curve.png"
        g["EQUITY_LOG_LATEST"] = latest_dir / "equity_log.csv"
        g["TRADE_LOG_LATEST"] = latest_dir / "trade_log_master.csv"

        g["EQUITY_LOG_THROTTLE_SEC"] = int(self.EQUITY_LOG_THROTTLE_SEC)
        g["SKIP_EQUITY_WHEN_DRY_RUN"] = bool(self.SKIP_EQUITY_WHEN_DRY_RUN)
        g["_LAST_EQUITY_LOG_TS"] = 0
        g["_TRADE_EVENT_FLAG"] = False

        g["MAX_DAILY_DRAWDOWN_PCT"] = float(self.MAX_DAILY_DRAWDOWN_PCT)
        g["KILL_SWITCH_COOLDOWN_MIN"] = int(self.KILL_SWITCH_COOLDOWN_MIN)
        g["EXIT_AFTER_CLOSE"] = bool(self.EXIT_AFTER_CLOSE)

        g["FLATTEN_INTO_CLOSE"] = bool(self.FLATTEN_INTO_CLOSE)
        g["FORCE_FIRST_BUY"] = bool(self.FORCE_FIRST_BUY)
        g["FORCE_FLATTEN_ON_EXIT"] = bool(self.FORCE_FLATTEN_ON_EXIT)

        g["DATA_TIMEFRAME"] = str(self.DATA_TIMEFRAME)
        os.environ["EXIT_AFTER_CLOSE"] = "1" if self.EXIT_AFTER_CLOSE else "0"

        os.environ["APCA_API_BASE_URL"] = self.APCA_API_BASE_URL
        os.environ["DRY_RUN"] = "1" if self.DRY_RUN else "0"
        os.environ["AUTO_RUN_LIVE"] = "1" if self.AUTO_RUN_LIVE else "0"
        os.environ["BARS_FEED"] = self.BARS_FEED

def configure_knobs(overrides: Mapping[str, object] = None) -> Knobs:
    defaults = Knobs(
        TICKERS=_to_list_csv(os.getenv("TICKERS", "UNH,GE")),
        ARTIFACTS_DIR=os.getenv("ARTIFACTS_DIR", str(PROJECT_ROOT / "artifacts")),
        RESULTS_ROOT=os.getenv("RESULTS_ROOT", str(PROJECT_ROOT / "results")),
        DATA_TIMEFRAME=os.getenv("DATA_TIMEFRAME", "1H"),
    )
    cfg = Knobs.from_env(defaults, PROJECT_ROOT, os.environ, overrides=overrides)
    cfg.apply_to_globals()
    return cfg

# ============================================================
# Time helpers
# ============================================================

def ensure_utc(ts_like) -> pd.Timestamp:
    """Return a timezone-aware UTC Timestamp from any datetime-like input."""
    ts = pd.Timestamp(ts_like)
    if ts.tzinfo is None:
        return ts.tz_localize("UTC")
    return ts.tz_convert("UTC")

def now_utc() -> datetime:
    return datetime.now(timezone.utc)

def utc_ts(dt_like) -> int:
    if isinstance(dt_like, (int, np.integer)):
        return int(dt_like)
    if isinstance(dt_like, (float, np.floating)):
        return int(dt_like)
    ts = ensure_utc(dt_like)
    return int(ts.value // 10**9)


def utcnow_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def _sleep_to_next_minute_block(n: int):
    n = max(1, int(n))
    now = now_utc()
    base = now.replace(second=0, microsecond=0)
    remainder = base.minute % n
    add = (n - remainder) % n
    if add == 0:
        add = n
    next_slot = base + timedelta(minutes=add)
    time.sleep(max(0.0, (next_slot - now).total_seconds()))

# ============================================================
# CSV logging (master) + equity snapshots
# ============================================================

TRADE_FIELDS = ["datetime_utc", "ticker", "signal", "action", "price", "equity", "qty", "comment"]

def ensure_trade_log_header():
    if (not TRADE_LOG_CSV.exists()) or (TRADE_LOG_CSV.stat().st_size == 0):
        pd.DataFrame(columns=TRADE_FIELDS).to_csv(TRADE_LOG_CSV, index=False)

def log_trade(
    ticker: str,
    signal: float,
    action: str,
    price: float,
    equity: float,
    qty: float = None,
    comment: str = "",
):
    ensure_trade_log_header()
    row = {
        "datetime_utc": utcnow_iso(),
        "ticker": ticker,
        "signal": int(signal) if signal is not None else "",
        "action": action,
        "price": (float(price) if price is not None and np.isfinite(price) else ""),
        "equity": (float(equity) if equity is not None and np.isfinite(equity) else ""),
        "qty": (float(qty) if qty is not None and np.isfinite(qty) else ""),
        "comment": (str(comment) if comment else ""),
    }
    with TRADE_LOG_CSV.open("a", newline="", encoding="utf-8") as f:
        csv.DictWriter(f, fieldnames=TRADE_FIELDS).writerow(row)
    try:
        shutil.copy2(TRADE_LOG_CSV, TRADE_LOG_LATEST)
    except Exception:
        pass

def init_alpaca() -> "tradeapi.REST":
    if not (globals().get("API_KEY") and globals().get("API_SECRET")):
        raise RuntimeError("Missing Alpaca API keys (check your .env).")
    return tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)

# Timeout-safe Alpaca calls for portfolio history
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FuturesTimeoutError
_TIMEOUT_EXEC = ThreadPoolExecutor(max_workers=8)

def _call_with_timeout(func, timeout_sec: int, *args, **kwargs):
    fut = _TIMEOUT_EXEC.submit(func, *args, **kwargs)
    try:
        return fut.result(timeout=timeout_sec)
    except FuturesTimeoutError:
        raise TimeoutError(f"Timed out after {timeout_sec}s")

def get_portfolio_history_safe(api, period="1M", timeframe=None, timeout_sec: int = 8, retries: int = 1):
    timeframe = timeframe or os.getenv("EQUITY_TIMEFRAME", os.getenv("DATA_TIMEFRAME", "1H"))
    last_exc = None
    for _ in range(max(1, retries + 1)):
        try:
            return _call_with_timeout(api.get_portfolio_history, timeout_sec, period=period, timeframe=timeframe)
        except Exception as e:
            last_exc = e
            time.sleep(0.5)
    logging.warning(f"get_portfolio_history_safe failed: {last_exc}")
    return None

def fetch_portfolio_history(period="1M", timeframe=None, api_in=None):
    timeframe = timeframe or os.getenv("EQUITY_TIMEFRAME", os.getenv("DATA_TIMEFRAME", "1H"))
    a = api_in if api_in is not None else globals().get("api", None)
    if a is None:
        return pd.DataFrame(columns=["timestamp_utc", "equity"])

    hist = get_portfolio_history_safe(
        a, period=period, timeframe=timeframe,
        timeout_sec=int(os.getenv("PH_TIMEOUT_SEC", "8")), retries=1
    )

    if (not hist) or (not getattr(hist, "timestamp", None)) or (not getattr(hist, "equity", None)):
        if EQUITY_LOG_CSV.exists():
            try:
                df = pd.read_csv(EQUITY_LOG_CSV, parse_dates=["datetime_utc"])
                return df.rename(columns={"datetime_utc": "timestamp_utc"})[["timestamp_utc", "equity"]]
            except Exception:
                pass
        return pd.DataFrame(columns=["timestamp_utc", "equity"])

    return pd.DataFrame({
        "timestamp_utc": pd.to_datetime(hist.timestamp, unit="s", utc=True),
        "equity": pd.Series(hist.equity, dtype="float64"),
    }).dropna()

def log_equity_snapshot(api_in=None):
    snap = fetch_portfolio_history(period="1D", timeframe=os.getenv("EQUITY_TIMEFRAME", "5Min"), api_in=api_in)
    if snap.empty:
        return
    latest = snap.iloc[-1:].copy().rename(columns={"timestamp_utc": "datetime_utc"})

    if EQUITY_LOG_CSV.exists():
        df_old = pd.read_csv(EQUITY_LOG_CSV, parse_dates=["datetime_utc"])
        if not df_old.empty and pd.to_datetime(df_old["datetime_utc"].iloc[-1]) == latest["datetime_utc"].iloc[0]:
            return
        pd.concat([df_old, latest], ignore_index=True)\
            .drop_duplicates(subset=["datetime_utc"], keep="last")\
            .to_csv(EQUITY_LOG_CSV, index=False)
    else:
        latest.to_csv(EQUITY_LOG_CSV, index=False)

    try:
        shutil.copy2(EQUITY_LOG_CSV, EQUITY_LOG_LATEST)
    except Exception:
        pass

def maybe_log_equity_snapshot(api_in=None, reason: str = "cycle"):
    global _LAST_EQUITY_LOG_TS, _TRADE_EVENT_FLAG
    if bool(globals().get("DRY_RUN", False)) and bool(globals().get("SKIP_EQUITY_WHEN_DRY_RUN", True)):
        return

    now_ts = time.time()
    force = reason in {"trade", "finalize", "close"}
    if force or (now_ts - float(_LAST_EQUITY_LOG_TS)) >= int(globals().get("EQUITY_LOG_THROTTLE_SEC", 900)):
        try:
            log_equity_snapshot(api_in=api_in)
            _LAST_EQUITY_LOG_TS = now_ts
        except Exception as e:
            logging.debug(f"maybe_log_equity_snapshot failed: {e}")

    if reason == "trade":
        _TRADE_EVENT_FLAG = False

def plot_equity_curve(from_equity_csv: bool = True):
    with plt.ioff():
        if from_equity_csv and EQUITY_LOG_CSV.exists():
            df = pd.read_csv(EQUITY_LOG_CSV, parse_dates=["datetime_utc"]).sort_values("datetime_utc")
        else:
            df = fetch_portfolio_history(period="3M", timeframe=os.getenv("EQUITY_TIMEFRAME", "5Min"))\
                .rename(columns={"timestamp_utc": "datetime_utc"})

        if df.empty:
            print("No equity data to plot yet.")
            return

        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(df["datetime_utc"], df["equity"])
        ax.set_title("Portfolio Value Over Time (Paper)")
        ax.set_xlabel("Time (UTC)")
        ax.set_ylabel("Equity ($)")
        fig.tight_layout()
        fig.savefig(PLOT_PATH, bbox_inches="tight")
        fig.savefig(PLOT_PATH_LATEST, bbox_inches="tight")
        plt.close(fig)
        print(f"Saved equity curve → {PLOT_PATH}")
        print(f"Updated latest copy → {PLOT_PATH_LATEST}")

def compute_performance_metrics(df_equity: pd.DataFrame):
    if df_equity.empty or df_equity["equity"].isna().all():
        return {"cum_return": np.nan, "sharpe": np.nan, "max_drawdown": np.nan}

    df = df_equity.sort_values("datetime_utc")
    e = df["equity"].astype(float)
    r = e.pct_change().dropna()
    if r.empty:
        return {"cum_return": 0.0, "sharpe": np.nan, "max_drawdown": np.nan}

    dt_sec = df["datetime_utc"].diff().dt.total_seconds().dropna().median()
    if not (isinstance(dt_sec, (int, float)) and dt_sec > 0):
        periods_per_year = 252 * 78  # fallback for ~5-min bars
    else:
        periods_per_day = (6.5 * 3600) / dt_sec
        periods_per_year = 252 * periods_per_day

    sharpe = (r.mean() / (r.std() + 1e-12)) * math.sqrt(periods_per_year)
    cum = (1 + r).cumprod()
    peak = cum.cummax()
    dd = (cum / peak - 1.0).min()
    cum_return = e.iloc[-1] / e.iloc[0] - 1.0
    return {"cum_return": float(cum_return), "sharpe": float(sharpe), "max_drawdown": float(dd)}

# ============================================================
# Per-ticker CSV logging
# ============================================================

def _append_csv_row(path: Path, row: dict):
    if path.name in ("trade_log_master.csv", "equity_log.csv"):
        raise RuntimeError(f"_append_csv_row must not write to master file: {path}")

    fieldnames = list(row.keys())

    if not path.exists():
        with path.open("w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=fieldnames)
            w.writeheader()
            w.writerow(row)
        return

    try:
        with path.open("r", newline="", encoding="utf-8") as f:
            r = csv.reader(f)
            old_header = next(r)
    except Exception:
        old_header = []

    if old_header != fieldnames:
        tmp = path.with_suffix(".tmp")
        with tmp.open("w", newline="", encoding="utf-8") as wf, path.open("r", newline="", encoding="utf-8") as rf:
            r = csv.DictReader(rf) if old_header else None
            w = csv.DictWriter(wf, fieldnames=fieldnames)
            w.writeheader()
            if r:
                for old_row in r:
                    merged = {k: old_row.get(k, "") for k in fieldnames}
                    w.writerow(merged)
        tmp.replace(path)

    with path.open("a", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writerow(row)

def log_trade_symbol(
    symbol: str,
    bar_time,
    signal: int,
    raw_action: float,
    weight: float,
    confidence: float,
    price: float,
    equity: float,
    dry_run: bool,
    note: str = "",
    order_submitted: int = 0,
    order_id: str = "",
    order_status: str = "",
    filled_qty: str = "",
):
    try:
        bt = pd.Timestamp(bar_time)
        if bt.tzinfo is None:
            bt = bt.tz_localize("UTC")
        else:
            bt = bt.tz_convert("UTC")
        bt_iso = bt.isoformat()
        age_sec = int((now_utc() - bt.to_pydatetime()).total_seconds())
    except Exception:
        bt_iso = ""
        age_sec = ""

    resolved_feed = (os.getenv("BARS_FEED", "").strip() or "default")

    if note:
        decision = note
    else:
        if abs(float(weight)) <= float(globals().get("EXIT_WEIGHT_MAX", 0.0)):
            decision = "hold_or_flat"
        else:
            decision = "rebalance_long" if float(weight) > 0 else "rebalance_short"

    try:
        w = float(weight) if weight is not None else 0.0
    except Exception:
        w = 0.0

    if abs(w) <= float(globals().get("EXIT_WEIGHT_MAX", 0.0)):
        sig = "FLAT"
    elif w > 0:
        sig = "LONG"
    else:
        sig = "SHORT"

    row = {
        "log_time": now_utc().isoformat(),
        "symbol": symbol,
        "bar_time": bt_iso,
        "bar_age_sec": age_sec,
        "feed": resolved_feed,
        "signal": sig,
        "raw_action": float(raw_action) if raw_action is not None and np.isfinite(raw_action) else "",
        "weight": float(weight) if weight is not None and np.isfinite(weight) else "",
        "confidence": float(confidence) if confidence is not None and np.isfinite(confidence) else "",
        "price": float(price) if price is not None and np.isfinite(price) else "",
        "equity": float(equity) if equity is not None and np.isfinite(equity) else "",
        "dry_run": int(bool(dry_run)),
        "decision": decision,
        "note": note,
        "order_submitted": int(order_submitted),
        "order_id": str(order_id or ""),
        "order_status": str(order_status or ""),
        "filled_qty": str(filled_qty or ""),
    }

    _append_csv_row(RESULTS_DIR / f"trade_log_{symbol}.csv", row)

    try:
        action = str(decision)[:64]
        comment = str(note or decision)[:200]
        log_trade(
            ticker=symbol,
            signal=1 if int(signal) == 1 else 0,
            action=action,
            price=float(price) if (price is not None and np.isfinite(price)) else None,
            equity=float(equity) if (equity is not None and np.isfinite(equity)) else None,
            qty=None,
            comment=comment,
        )
    except Exception as e:
        logging.debug("master trade log write failed: %s", e)

# ============================================================
# Artifacts: picker & loaders
# ============================================================

def _extract_window_idx(path: Path) -> Optional[int]:
    m = re.search(r"_window(\d+)", path.stem)
    return int(m.group(1)) if m else None

def _prefer_same_window(cands, w: Optional[int]):
    cands = list(cands)
    if not cands:
        return []
    if w is None:
        return sorted(cands)
    same = [p for p in cands if _extract_window_idx(p) == w]
    return sorted(same or cands)

def pick_artifacts_for_ticker(ticker: str, artifacts_dir: str, best_window: Optional[str] = None) -> Dict[str, Optional[Path]]:
    p = Path(artifacts_dir)
    if not p.exists():
        raise FileNotFoundError(f"Artifacts directory not found: {p.resolve()}")

    models = sorted(p.glob(f"ppo_{ticker}_window*_model*.zip"))
    if not models:
        models = (
            sorted(p.glob(f"ppo_{ticker}_model*.zip")) or
            sorted(p.glob(f"*{ticker}*model*.zip"))
        )
    if not models:
        raise FileNotFoundError(f"No PPO model zip found for {ticker} in {p}")

    def _model_sort_key(path: Path):
        w = _extract_window_idx(path)
        return (w if w is not None else -1, " (1)" in path.stem)

    models = sorted(models, key=_model_sort_key)

    chosen: Optional[Path] = None
    if best_window:
        chosen = next((m for m in models if f"_window{best_window}_" in m.stem), None)
        if chosen is None:
            logging.warning("[%s] BEST_WINDOW=%s not found; falling back.", ticker, best_window)

    if chosen is None:
        with_idx = [(m, _extract_window_idx(m)) for m in models]
        with_idx = [(m, w) for (m, w) in with_idx if w is not None]
        chosen = max(with_idx, key=lambda t: t[1])[0] if with_idx else models[-1]

    chosen_w = _extract_window_idx(chosen)

    vec_candidates = list(p.glob(f"ppo_{ticker}_window*_vecnorm*.pkl"))
    feat_candidates = list(p.glob(f"ppo_{ticker}_window*_features*.json"))

    vecnorm = (_prefer_same_window(vec_candidates, chosen_w)[0] if vec_candidates else None)
    feats = (_prefer_same_window(feat_candidates, chosen_w)[0] if feat_candidates else None)

    logging.info("[%s] model=%s | window=%s | vecnorm=%s | features=%s",
                 ticker,
                 chosen.name,
                 chosen_w,
                 vecnorm.name if vecnorm else "None",
                 feats.name if feats else "None")

    return {"model": chosen, "vecnorm": vecnorm, "features": feats}

def load_vecnormalize(path: Optional[Path]):
    if path is None:
        return None

    try:
        with open(path, "rb") as f:
            return pickle.load(f)
    except Exception:
        pass

    try:
        return VecNormalize.load(str(path), venv=None)
    except Exception as e:
        logging.warning("VecNormalize load failed (%s). Proceeding without normalization.", e)
        return None

def load_features(path: Optional[Path]):
    if path is None:
        return None
    with open(path, "r") as f:
        return json.load(f)

def _const_schedule(val: float):
    # SB3 schedules are callables: f(progress_remaining) -> float
    return lambda _progress_remaining: float(val)

def load_ppo_model(model_path: Path):
    # Pick safe defaults; these only matter if the model tries to use them during training.
    # For inference, they’re irrelevant—but this prevents the unpickle warning.
    custom_objects = {
        "lr_schedule": _const_schedule(5e-5),
        "clip_range":  _const_schedule(0.2),
        # Sometimes also present depending on SB3 version / PPO config:
        "clip_range_vf": _const_schedule(0.2),
    }
    return PPO.load(str(model_path), custom_objects=custom_objects)

# Cached asset flags
@lru_cache(maxsize=256)
def _asset_flags(symbol: str) -> Tuple[bool, bool, bool]:
    try:
        _api = globals().get("api") or init_alpaca()
        a = _api.get_asset(symbol)
        return (
            bool(getattr(a, "tradable", True)),
            bool(getattr(a, "fractionable", False)),
            bool(getattr(a, "shortable", False)),
        )
    except Exception:
        return True, False, False

def _can_seed_short(api, symbol: str) -> Tuple[bool, str]:
    if not globals().get("ALLOW_SHORTS", False):
        return False, "shorts_disabled_seed"
    try:
        a = api.get_asset(symbol)
        if not getattr(a, "shortable", False):
            return False, "not_shortable_seed"
        return True, ""
    except Exception as e:
        logging.info(f"[{symbol}] get_asset shortable check failed: {e}")
        return False, "shortable_check_error"

# ============================================================
# Market data + account helpers
# ============================================================

def get_recent_bars(api, symbol: str, limit: int = 200, timeframe=LIVE_TIMEFRAME) -> pd.DataFrame:
    def _as_df(bars):
        if hasattr(bars, "df"):
            df = bars.df.copy()
            if not df.empty:
                if isinstance(df.index, pd.MultiIndex):
                    try:
                        df = df.xs(symbol, level=0)
                    except KeyError:
                        df = df.reset_index(level=0, drop=True)
                df.index = pd.to_datetime(df.index, utc=True, errors="coerce")
                df = df.rename(columns={"open": "Open", "high": "High", "low": "Low", "close": "Close", "volume": "Volume"})
                cols = [c for c in ["Open", "High", "Low", "Close", "Volume"] if c in df.columns]
                return df[cols].sort_index()
            return pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume"])

        rows = []
        for b in bars:
            ts = getattr(b, "t", None)
            ts = pd.to_datetime(ts, utc=True) if ts is not None else pd.NaT
            rows.append({
                "timestamp": ts,
                "Open": float(getattr(b, "o", getattr(b, "open", np.nan))),
                "High": float(getattr(b, "h", getattr(b, "high", np.nan))),
                "Low": float(getattr(b, "l", getattr(b, "low", np.nan))),
                "Close": float(getattr(b, "c", getattr(b, "close", np.nan))),
                "Volume": float(getattr(b, "v", getattr(b, "volume", np.nan))),
            })
        df = pd.DataFrame(rows)
        if df.empty:
            return pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume"])
        return df.set_index(pd.to_datetime(df["timestamp"], utc=True)).drop(columns=["timestamp"]).sort_index()

    feed = os.getenv("BARS_FEED", "").strip()
    try:
        logging.info(f"[{symbol}] fetching {limit} {timeframe} bars (feed='{feed or 'default'}')")
        bars = api.get_bars(symbol, timeframe, limit=limit, feed=feed) if feed else api.get_bars(symbol, timeframe, limit=limit)
        df = _as_df(bars)
        if not df.empty:
            return df
        if feed:
            logging.info(f"[{symbol}] explicit feed empty; retrying with default feed")
            df2 = _as_df(api.get_bars(symbol, timeframe, limit=limit))
            if not df2.empty:
                return df2
    except Exception as e:
        logging.warning(f"[{symbol}] get_bars(limit) failed: {e}")

    try:
        end_dt = datetime.now(timezone.utc).replace(microsecond=0)
        start_dt = end_dt - timedelta(days=5)
        end = end_dt.isoformat().replace("+00:00", "Z")
        start = start_dt.isoformat().replace("+00:00", "Z")
        logging.info(f"[{symbol}] retry window start={start} end={end} (feed='{feed or 'default'}')")
        bars = api.get_bars(symbol, timeframe, start=start, end=end, feed=feed) if feed else api.get_bars(symbol, timeframe, start=start, end=end)
        return _as_df(bars)
    except Exception as e:
        logging.warning(f"[{symbol}] get_bars(start/end) failed: {e}")
        return pd.DataFrame(columns=["Open", "High", "Low", "Close", "Volume"])

def get_account_equity(api) -> float:
    return float(api.get_account().equity)

def get_position(api, symbol: str):
    try:
        return api.get_position(symbol)
    except Exception:
        return None

def get_position_qty(api, symbol: str):
    try:
        pos = api.get_position(symbol)
    except Exception:
        pos = None
    if not pos:
        return 0.0 if USE_FRACTIONALS else 0
    try:
        q = float(pos.qty)
        return q if USE_FRACTIONALS else int(round(q))
    except Exception:
        return 0.0 if USE_FRACTIONALS else 0

def get_last_price(api, symbol: str) -> float:
    try:
        tr = api.get_latest_trade(symbol)
        price = getattr(tr, "price", None)
        if price is None:
            price = getattr(tr, "p", None)
        if price is not None and np.isfinite(price):
            return float(price)
    except Exception:
        pass

    try:
        feed = os.getenv("BARS_FEED", "").strip() or None
        bars = api.get_bars(symbol, LIVE_TIMEFRAME, limit=1, feed=feed) if feed else api.get_bars(symbol, LIVE_TIMEFRAME, limit=1)
        if hasattr(bars, "df"):
            df = bars.df.copy()
            if isinstance(df.index, pd.MultiIndex):
                try:
                    df = df.xs(symbol, level=0)
                except Exception:
                    df = df.reset_index(level=0, drop=True)
            if not df.empty:
                if "close" in df.columns:
                    return float(df["close"].iloc[-1])
                if "Close" in df.columns:
                    return float(df["Close"].iloc[-1])
        elif bars:
            b = bars[0]
            close = getattr(b, "c", getattr(b, "close", None))
            if close is not None:
                return float(close)
    except Exception as e:
        logging.warning(f"[{symbol}] get_last_price via bars failed: {e}")

    try:
        qt = api.get_latest_quote(symbol)
        ap = getattr(qt, "ap", None) or getattr(qt, "ask_price", None)
        bp = getattr(qt, "bp", None) or getattr(qt, "bid_price", None)
        if ap and bp:
            return float((float(ap) + float(bp)) / 2.0)
        if ap:
            return float(ap)
        if bp:
            return float(bp)
    except Exception:
        pass

    try:
        pos = api.get_position(symbol)
        return float(pos.avg_entry_price)
    except Exception:
        return float("nan")

def flatten_symbol(api, symbol: str):
    try:
        api.close_position(symbol)
        logging.info(f"[{symbol}] close_position submitted.")
    except Exception as e:
        logging.warning(f"[{symbol}] close_position failed: {e}")

def flatten_all_positions(api):
    try:
        api.close_all_positions()
        logging.info("close_all_positions submitted.")
    except Exception as e:
        logging.warning(f"close_all_positions failed: {e}")

def to_2dp_str(x) -> str:
    return format(Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP), "f")

def to_6dp_str(x) -> str:
    return format(Decimal(str(x)).quantize(Decimal("0.000001"), rounding=ROUND_DOWN), "f")

def market_order(api, symbol: str, side: str, qty=None, notional: float = None):
    if qty is not None and notional is not None:
        logging.warning(f"[{symbol}] Both qty and notional provided; preferring notional.")
        qty = None

    if qty is None and notional is None:
        logging.warning(f"[{symbol}] No order size provided; skipping.")
        return None

    if qty is not None:
        try:
            if float(qty) <= 0:
                logging.warning(f"[{symbol}] Non-positive qty ({qty}); skipping.")
                return None
        except Exception:
            pass
    if notional is not None and notional <= 0:
        logging.warning(f"[{symbol}] Non-positive notional (${notional}); skipping.")
        return None

    try:
        if (notional is not None and (not np.isfinite(float(notional)) or float(notional) < 0.01)) or \
           (qty is not None and (not np.isfinite(float(qty)) or float(qty) == 0.0)):
            logging.info(f"[{symbol}] Order size ~0; skipping.")
            return None
    except Exception:
        logging.info(f"[{symbol}] Order size parse issue; skipping.")
        return None

    if DRY_RUN:
        notional_str = to_2dp_str(notional) if notional is not None else None
        logging.info(
            f"[DRY_RUN] Would submit {side} "
            f"{('notional=$' + str(notional_str)) if notional_str is not None else ('qty=' + str(qty))} "
            f"{symbol} (market, day)"
        )
        globals()["_TRADE_EVENT_FLAG"] = True
        return None

    try:
        qty_arg = None
        if qty is not None:
            qty_arg = to_6dp_str(float(qty)) if USE_FRACTIONALS else int(qty)
            if (not USE_FRACTIONALS) and int(qty_arg) <= 0:
                logging.info(f"[{symbol}] qty rounds to 0 shares; skipping.")
                return None
        notional_arg = to_2dp_str(float(notional)) if notional is not None else None

        o = api.submit_order(
            symbol=symbol,
            side=side,
            type="market",
            time_in_force="day",
            qty=qty_arg,
            notional=notional_arg,
        )

        logging.info(
            f"[{symbol}] Submitted {side} "
            f"{('notional=$' + str(notional_arg)) if notional_arg is not None else ('qty=' + str(qty_arg))}"
        )
        globals()["_TRADE_EVENT_FLAG"] = True
        return o

    except Exception as e:
        logging.error(f"[{symbol}] submit_order failed: {e}")
        return None

def market_order_to_qty(api, symbol: str, side: str, qty):
    if qty is None:
        logging.warning(f"[{symbol}] qty is None; skipping.")
        return None

    try:
        q = float(qty)
    except Exception:
        logging.warning(f"[{symbol}] qty not numeric ({qty}); skipping.")
        return None

    if not np.isfinite(q) or q <= 0:
        logging.info(f"[{symbol}] Non-positive qty ({qty}); skipping.")
        return None

    if not bool(globals().get("USE_FRACTIONALS", True)):
        q_int = int(math.floor(q))
        if q_int <= 0:
            logging.info(f"[{symbol}] qty rounds to 0 shares; skipping.")
            return None
        q = q_int

    if bool(globals().get("DRY_RUN", False)):
        logging.info(f"[DRY_RUN] Would submit {side} qty={q} {symbol} (market, day)")
        globals()["_TRADE_EVENT_FLAG"] = True
        return None

    if side == "sell" and (not can_short_symbol(api, symbol)):
        have_qty = get_position_qty(api, symbol)
        if have_qty <= 0:
            logging.info(f"[{symbol}] Sell skipped (no shares and shorting not allowed).")
            return None
        q = min(float(q), float(have_qty))
        if q <= 0:
            logging.info(f"[{symbol}] Sell qty clamped to 0; skipping.")
            return None

    try:
        qty_arg = to_6dp_str(float(q)) if bool(globals().get("USE_FRACTIONALS", True)) else int(q)
        o = api.submit_order(
            symbol=symbol,
            side=side,
            type="market",
            time_in_force="day",
            qty=qty_arg,
        )
        logging.info(f"[{symbol}] Submitted {side} qty={qty_arg}")
        globals()["_TRADE_EVENT_FLAG"] = True
        return o
    except Exception as e:
        logging.error(f"[{symbol}] submit_order(qty) failed: {e}")
        return None

def submit_fractional_rebalance(api, symbol: str, delta_notional: float, price: float):
    dn = round_to_cents(abs(delta_notional))
    if dn < float(globals().get("REBALANCE_MIN_NOTIONAL", 0.0)):
        return None

    if delta_notional > 0:
        return market_order(api, symbol, side="buy", notional=dn)
    qty = dn / max(float(price), 1e-9)

    if not can_short_symbol(api, symbol):
        have_qty = get_position_qty(api, symbol)
        if have_qty <= 0:
            logging.info(f"[{symbol}] Fractional sell skipped (no shares, no shorting).")
            return None
        qty = min(float(qty), float(have_qty))
        if qty <= 0:
            return None
    return market_order_to_qty(api, symbol, side="sell", qty=qty)

NO_ORDER = {"order_submitted": 0, "order_id": "", "order_status": "", "filled_qty": ""}

def _order_info(order_obj) -> dict:
    if order_obj is None:
        return dict(NO_ORDER)
    return {
        "order_submitted": 1,
        "order_id": str(getattr(order_obj, "id", "") or ""),
        "order_status": str(getattr(order_obj, "status", "") or ""),
        "filled_qty": str(getattr(order_obj, "filled_qty", "") or ""),
    }

def compute_target_qty_by_cash(equity: float, price: float, target_weight: float, api=None) -> int:
    if not np.isfinite(equity) or equity <= 0:
        return 0
    if not np.isfinite(price) or price <= 0:
        return 0

    w = float(target_weight)
    cap = float(globals().get("WEIGHT_CAP", 1.0))
    if cap > 0:
        w = max(-cap, min(cap, w))
    if not bool(globals().get("ALLOW_SHORTS", False)):
        w = max(0.0, w)

    target_notional = equity * w

    if target_notional >= 0:
        qty = int(math.floor(target_notional / price))
    else:
        qty = int(math.ceil(target_notional / price))
    return qty

def rebalance_to_weight(api, symbol: str, equity: float, target_weight: float) -> dict:
    price = get_last_price(api, symbol)
    if not np.isfinite(price) or price <= 0:
        logging.warning(f"[{symbol}] Price unavailable; skipping rebalance.")
        return dict(NO_ORDER)

    tradable, fractionable, shortable = _asset_flags(symbol)
    if not tradable:
        logging.info(f"[{symbol}] Not tradable; skipping.")
        return dict(NO_ORDER)

    use_fractionals = bool(USE_FRACTIONALS and fractionable)

    have_qty = get_position_qty(api, symbol)
    have_notional = have_qty * price
    target_notional = equity * float(target_weight)
    delta_notional = target_notional - have_notional

    if have_qty > 0 and target_notional < 0:
      logging.info(f"[{symbol}] Flip long→short requested. Flattening long first (have_qty={have_qty}).")
      flatten_symbol(api, symbol)
      return dict(NO_ORDER)  # next cycle will open the short cleanly

    if abs(delta_notional) < 1e-9:
        return dict(NO_ORDER)

    delta_weight = abs(delta_notional) / max(float(equity), 1e-9)
    if delta_weight < float(globals().get("DELTA_WEIGHT_MIN", 0.0)):
        return dict(NO_ORDER)

    if use_fractionals:
        dn = round_to_cents(abs(delta_notional))
        if dn < float(globals().get("REBALANCE_MIN_NOTIONAL", 0.0)):
            return dict(NO_ORDER)

        side = "buy" if delta_notional > 0 else "sell"
        shorting = (target_notional < 0) and (side == "sell")
        covering = (have_qty < 0) and (side == "buy")

        if shorting:
            if not shortable:
                logging.info(f"[{symbol}] Not shortable; skipping short rebalance.")
                return dict(NO_ORDER)
            qty = max(1, int(math.floor(dn / price))) if price > 0 else 1
            o = market_order_to_qty(api, symbol, side="sell", qty=qty)
            return _order_info(o)

        if covering:
            qty = max(1, int(math.ceil(dn / price))) if price > 0 else 1
            qty = min(int(abs(have_qty)), qty) if have_qty < 0 else qty
            o = market_order_to_qty(api, symbol, side="buy", qty=qty)
            return _order_info(o)

        o = submit_fractional_rebalance(api, symbol, delta_notional=delta_notional, price=price)
        return _order_info(o)

    want_qty = compute_target_qty_by_cash(equity, price, target_weight, api)
    delta_qty = want_qty - have_qty
    if delta_qty == 0:
        return dict(NO_ORDER)

    approx_delta_notional = abs(delta_qty) * price
    if equity > 0 and approx_delta_notional / equity < float(globals().get("DELTA_WEIGHT_MIN", 0.0)):
        return dict(NO_ORDER)
    if approx_delta_notional < float(globals().get("REBALANCE_MIN_NOTIONAL", 0.0)):
        return dict(NO_ORDER)

    side = "buy" if delta_qty > 0 else "sell"
    shorting = (target_notional < 0) and (side == "sell")
    if shorting and not shortable:
        logging.info(f"[{symbol}] Not shortable; skipping short rebalance.")
        return dict(NO_ORDER)

    o = market_order_to_qty(api, symbol, side=side, qty=int(abs(delta_qty)))
    return _order_info(o)

def check_tp_sl_and_maybe_flatten(api, symbol: str) -> bool:
    if TAKE_PROFIT_PCT <= 0 and STOP_LOSS_PCT <= 0:
        return False
    pos = get_position(api, symbol)
    if not pos:
        return False
    try:
        plpc = float(pos.unrealized_plpc)
    except Exception:
        return False
    if TAKE_PROFIT_PCT > 0 and plpc >= TAKE_PROFIT_PCT:
        logging.info(f"[{symbol}] TP hit ({plpc:.4f} >= {TAKE_PROFIT_PCT:.4f}). Flattening.")
        flatten_symbol(api, symbol)
        return True
    if STOP_LOSS_PCT > 0 and plpc <= -abs(STOP_LOSS_PCT):
        logging.info(f"[{symbol}] SL hit ({plpc:.4f} <= {-abs(STOP_LOSS_PCT):.4f}). Flattening.")
        flatten_symbol(api, symbol)
        return True
    return False

# ============================================================
# Inference / obs building
# ============================================================

def expected_obs_shape(model, vecnorm) -> Optional[tuple]:
    for src in (vecnorm, model):
        try:
            shp = tuple(src.observation_space.shape)
            if shp:
                return shp
        except Exception:
            pass
    return None


# ---- Feature alias resolution (training ↔ live parity) ----
FEATURE_ALIASES = {
    "SMA_50": "Rolling_Mean_50",
    "Rolling_Mean_50": "SMA_50",
}

def resolve_feature_alias(name: str, df: pd.DataFrame) -> Optional[str]:
    if name in df.columns:
        return name
    alt = FEATURE_ALIASES.get(name)
    if alt and alt in df.columns:
        return alt
    return None


def compute_art_feat_order(features_hint: Any, df: pd.DataFrame) -> List[str]:
    if features_hint is None:
        return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

    feats = features_hint.get("features", features_hint) if isinstance(features_hint, dict) else list(features_hint)
    drop = {"datetime", "symbol", "target", "return"}

    resolved = []
    for f in feats:
        if f in drop:
            continue
        col = resolve_feature_alias(f, df)
        if col and pd.api.types.is_numeric_dtype(df[col]):
            resolved.append(col)

    return resolved

def build_obs_from_row(row: pd.Series, order: List[str]) -> np.ndarray:
    vals = []
    for c in order:
        v = row.get(c, np.nan)
        vals.append(0.0 if (pd.isna(v) or v is None or v is False) else float(v))
    return np.array(vals, dtype=np.float32)

def _pick_columns_for_channels(features_hint: Any, df: pd.DataFrame, channels: int) -> List[str]:
    numeric = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    cols: List[str] = []
    if isinstance(features_hint, dict) and "features" in features_hint:
        cand = [c for c in features_hint["features"] if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
        if len(cand) >= channels:
            cols = cand[:channels]
    if not cols:
        pref = ["Close", "Volume", "Adj Close", "Open", "High", "Low"]
        cols = [c for c in pref if c in numeric]
        cols += [c for c in numeric if c not in cols]
        cols = cols[:channels]
    if len(cols) < channels and cols:
        while len(cols) < channels:
            cols.append(cols[-1])
    return cols[:channels]

def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    df["Vol20"] = df["Close"].pct_change().rolling(20).std()
    df["Ret20"] = df["Close"].pct_change(20)
    vol_hi = (df["Vol20"] > df["Vol20"].median()).astype(int)
    trend_hi = (df["Ret20"].abs() > df["Ret20"].abs().median()).astype(int)
    df["Regime4"] = vol_hi * 2 + trend_hi
    return df

def denoise_wavelet(series: pd.Series, wavelet: str = "db1", level: int = 2) -> pd.Series:
    try:
        import pywt
    except Exception:
        return pd.Series(series).astype(float).ffill().bfill().ewm(span=5, adjust=False).mean()

    s = pd.Series(series).astype(float).ffill().bfill()
    arr = s.to_numpy()
    try:
        w = pywt.Wavelet(wavelet)
        maxlvl = pywt.dwt_max_level(len(arr), w.dec_len)
        lvl = int(max(0, min(level, maxlvl)))
        if lvl < 1:
            return s
        coeffs = pywt.wavedec(arr, w, mode="symmetric", level=lvl)
        for i in range(1, len(coeffs)):
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, w, mode="symmetric")
        return pd.Series(rec[:len(arr)], index=s.index)
    except Exception:
        return s.ewm(span=5, adjust=False).mean()

def add_features_live(
    df: pd.DataFrame,
    use_sentiment: bool = False,
    rsi_wilder: bool = True,
    atr_wilder: bool = True,
) -> pd.DataFrame:
    df = df.copy().sort_index()
    cols_ci = {c.lower(): c for c in df.columns}
    rename = {}
    for final, alts in {
        "Open": ["open"],
        "High": ["high"],
        "Low": ["low"],
        "Close": ["close", "close*", "last"],
        "Adj Close": ["adj close", "adj_close", "adjclose", "adjusted close"],
        "Volume": ["volume", "vol"],
    }.items():
        for a in [final.lower()] + alts:
            if a in cols_ci:
                rename[cols_ci[a]] = final
                break
    df = df.rename(columns=rename)
    if "Adj Close" not in df.columns and "Close" in df.columns:
        df["Adj Close"] = df["Close"]

    df["SMA_20"] = df["Close"].rolling(20).mean()
    df["STD_20"] = df["Close"].rolling(20).std()
    df["Upper_Band"] = df["SMA_20"] + 2 * df["STD_20"]
    df["Lower_Band"] = df["SMA_20"] - 2 * df["STD_20"]

    df["Lowest_Low"] = df["Low"].rolling(14).min()
    df["Highest_High"] = df["High"].rolling(14).max()
    denom = (df["Highest_High"] - df["Lowest_Low"]).replace(0, np.nan)
    df["Stoch"] = ((df["Close"] - df["Lowest_Low"]) / denom) * 100

    df["ROC"] = df["Close"].pct_change(10)
    sign = np.sign(df["Close"].diff().fillna(0))
    df["OBV"] = (sign * df["Volume"].fillna(0)).cumsum()

    tp = (df["High"] + df["Low"] + df["Close"]) / 3.0
    sma_tp = tp.rolling(20).mean()
    md = (tp - sma_tp).abs().rolling(20).mean().replace(0, np.nan)
    df["CCI"] = (tp - sma_tp) / (0.015 * md)

    df["EMA_10"] = df["Close"].ewm(span=10, adjust=False).mean()
    df["EMA_50"] = df["Close"].ewm(span=50, adjust=False).mean()
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD_Line"] = ema12 - ema26
    df["MACD_Signal"] = df["MACD_Line"].ewm(span=9, adjust=False).mean()

    d = df["Close"].diff()
    gain = d.clip(lower=0)
    loss = (-d.clip(upper=0))
    if rsi_wilder:
        avg_gain = gain.ewm(alpha=1 / 14, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1 / 14, adjust=False).mean()
    else:
        avg_gain = gain.rolling(14).mean()
        avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    df["RSI"] = 100 - (100 / (1 + rs))

    tr = pd.concat([
        (df["High"] - df["Low"]),
        (df["High"] - df["Close"].shift()).abs(),
        (df["Low"] - df["Close"].shift()).abs(),
    ], axis=1).max(axis=1)
    df["ATR"] = tr.ewm(alpha=1 / 14, adjust=False).mean() if atr_wilder else tr.rolling(14).mean()

    df["Volatility"] = df["Close"].pct_change().rolling(20).std()
    df["Denoised_Close"] = denoise_wavelet(df["Close"])

    df = add_regime(df)
    df["SentimentScore"] = (df.get("SentimentScore", 0.0) if use_sentiment else 0.0)
    df["Delta"] = df["Close"].pct_change(1).fillna(0.0)
    df["Gamma"] = df["Delta"].diff().fillna(0.0)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df

def prepare_observation_from_bars(
    bars_df: pd.DataFrame,
    features_hint: Any = None,
    min_required_rows: int = 60,
    expected_shape: Optional[tuple] = None,
    symbol: str = "",
) -> Tuple[np.ndarray, int]:

    feats_df = add_features_live(bars_df).replace([np.inf, -np.inf], np.nan)

    ts = ensure_utc(pd.Timestamp.utcnow())

    if not feats_df.empty:
        try:
            ts = ensure_utc(feats_df.index[-1])
        except Exception:
            pass

    obs_ts = int(ts.timestamp())

    if expected_shape is not None:
        if len(expected_shape) == 2:
            lookback, channels = int(expected_shape[0]), int(expected_shape[1])
            cols = _pick_columns_for_channels(features_hint, feats_df, channels)
            window_df = feats_df[cols].tail(lookback).fillna(0.0)
            arr = window_df.to_numpy(dtype=np.float32)
            if arr.shape[0] < lookback:
                pad_rows = lookback - arr.shape[0]
                arr = np.vstack([np.zeros((pad_rows, channels), dtype=np.float32), arr])
            arr = arr[-lookback:, :channels]
            return arr.reshape(lookback, channels), obs_ts

        elif len(expected_shape) == 1:
            n = int(expected_shape[0])
            cand = compute_art_feat_order(features_hint, feats_df)
            if len(feats_df) < max(20, min_required_rows):
                raise ValueError(f"Not enough bars to compute features robustly (have {len(feats_df)}).")
            last = feats_df.iloc[-1]
            vals = []
            for c in cand[:n]:
                v = last.get(c, np.nan)
                vals.append(0.0 if (pd.isna(v) or v is None) else float(v))
            if len(vals) < n:
                vals += [0.0] * (n - len(vals))
            return np.asarray(vals, dtype=np.float32), obs_ts

    order = compute_art_feat_order(features_hint, feats_df)
    missing = []
    if isinstance(features_hint, dict) and "features" in features_hint:
        missing = [c for c in features_hint["features"] if c not in feats_df.columns]

    logging.info("[%s] features_used=%d missing_from_live=%d", symbol, len(order), len(missing))
    if missing:
        logging.debug("[%s] missing examples: %s", symbol, missing[:20])

    if not order:
        raise ValueError("No usable features after resolving artifact order.")
    feats_df = feats_df.dropna(subset=order)
    if len(feats_df) < max(20, min_required_rows):
        raise ValueError(f"Not enough bars to compute features robustly (have {len(feats_df)}).")

    last = feats_df.iloc[-1]
    obs = build_obs_from_row(last, order)
    return obs.astype(np.float32), obs_ts

# ============================================================
# Live loop helpers
# ============================================================

def ensure_market_open(api) -> bool:
    try:
        return bool(api.get_clock().is_open)
    except Exception:
        return False

def _sleep_until_open(api):
    try:
        clock = api.get_clock()
        if getattr(clock, "is_open", False):
            return
        nxt = pd.to_datetime(getattr(clock, "next_open"), utc=True, errors="coerce")
        if pd.isna(nxt):
            time.sleep(60)
            return
        wait = max(1, int((nxt - now_utc()).total_seconds()))
        logging.info("Market closed. Sleeping %ds until next open.", wait)
        time.sleep(wait)
    except Exception:
        time.sleep(60)

def write_account_info_to_run_config(api) -> None:
    try:
        acct = api.get_account()
        acct_info = {
            "account_id": getattr(acct, "id", ""),
            "status": getattr(acct, "status", ""),
            "equity": getattr(acct, "equity", ""),
            "cash": getattr(acct, "cash", ""),
            "pattern_day_trader": getattr(acct, "pattern_day_trader", ""),
        }

        cfg_path = RESULTS_DIR / "run_config.json"
        try:
            meta = json.loads(cfg_path.read_text()) if cfg_path.exists() else {}
        except Exception:
            meta = {}

        meta["alpaca_account"] = acct_info
        tmp = cfg_path.with_suffix(".tmp")
        tmp.write_text(json.dumps(meta, indent=2))
        tmp.replace(cfg_path)

    except Exception as e:
        logging.warning("Could not augment run_config.json with account info: %s", e)

def action_to_weight(action) -> Tuple[float, float, float]:
    a = float(np.asarray(action).reshape(-1)[0])
    raw = a

    cap = float(globals().get("WEIGHT_CAP", 0.35))
    target_w = float(np.clip(a, -1, 1)) * cap
    conf = float(min(1.0, abs(a)))

    if not bool(globals().get("ALLOW_SHORTS", False)):
        target_w = max(0.0, target_w)

    if str(globals().get("SIZING_MODE", "linear")).lower() == "threshold":
        floor = float(globals().get("CONF_FLOOR", 0.15))
        if conf < floor:
            target_w = 0.0
        else:
            scale = (conf - floor) / max(1e-9, (1.0 - floor))
            target_w = np.sign(target_w) * cap * float(np.clip(scale, 0, 1))

    return float(target_w), float(conf), float(raw)

def infer_target_weight(model: PPO, vecnorm: Optional[VecNormalize], obs: np.ndarray) -> Tuple[float, float, float]:
    x = np.asarray(obs, dtype=np.float32)

    if vecnorm is not None and hasattr(vecnorm, "normalize_obs") and getattr(vecnorm, "obs_rms", None) is not None:
        try:
            x = vecnorm.normalize_obs(x)
        except Exception:
            try:
                x = vecnorm.normalize_obs(np.expand_dims(x, axis=0))[0]
            except Exception:
                pass

    try:
        action, _ = model.predict(x, deterministic=INF_DETERMINISTIC)
    except Exception:
        action, _ = model.predict(np.expand_dims(x, axis=0), deterministic=INF_DETERMINISTIC)
        if isinstance(action, (list, np.ndarray)):
            action = np.asarray(action)
            if action.ndim > 0:
                action = action[0]

    return action_to_weight(action)

def maybe_patch_stale_with_latest_trade(api, symbol: str, bars_df: pd.DataFrame, max_age_sec: int = None) -> pd.DataFrame:
    if bars_df.empty:
        return bars_df
    max_age_sec = max_age_sec or int(globals().get("STALE_MAX_SEC", 600))
    try:
        last_ts = pd.Timestamp(bars_df.index[-1])
        last_ts = last_ts.tz_convert("UTC") if last_ts.tzinfo else last_ts.tz_localize("UTC")
        age_sec = int((now_utc() - last_ts).total_seconds())
        if age_sec <= max_age_sec:
            return bars_df

        lt = api.get_latest_trade(symbol)
        price = float(getattr(lt, "price", getattr(lt, "p", float("nan"))))
        ts = pd.to_datetime(getattr(lt, "timestamp", getattr(lt, "t", None)), utc=True)
        if not (pd.notna(ts) and np.isfinite(price)):
            return bars_df

        lt_age = int((now_utc() - ts).total_seconds())
        if lt_age > max_age_sec:
            return bars_df

        synth_time = max(last_ts + pd.Timedelta(minutes=1), ts.floor("min"))
        row = pd.DataFrame(
            {"Open": [price], "High": [price], "Low": [price], "Close": [price], "Volume": [0.0]},
            index=pd.DatetimeIndex([synth_time], tz="UTC")
        )
        patched = pd.concat([bars_df, row]).sort_index()
        patched = patched[~patched.index.duplicated(keep="last")]
        logging.info(f"[{symbol}] Patched stale bars with synthetic trade bar @ {synth_time.isoformat()} px={price:.2f}")
        return patched
    except Exception as e:
        logging.debug(f"[{symbol}] maybe_patch_stale_with_latest_trade failed: {e}")
        return bars_df

def run_live_once_for_symbol(
    api,
    symbol: str,
    model: PPO,
    vecnorm: Optional[VecNormalize],
    features_hint: Optional[dict] = None,
    cycle_equity: Optional[float] = None,
):
    shape = expected_obs_shape(model, vecnorm)
    lookback = int(shape[0]) if (shape and len(shape) == 2) else None
    bars_need = max(200, (lookback or 0) * 3)

    bars_df = get_recent_bars(api, symbol, limit=bars_need, timeframe=LIVE_TIMEFRAME)
    if bars_df is None or bars_df.empty:
        logging.warning("[%s] No recent bars; skipping.", symbol)
        return

    bars_df = maybe_patch_stale_with_latest_trade(api, symbol, bars_df)

    block_until = _REENTRY_BLOCK_UNTIL.get(symbol, 0.0)
    if time.time() < block_until:
        remaining = int(max(0, block_until - time.time()))
        logging.info(f"[{symbol}] Re-entry cooldown active ({remaining}s left); skipping.")
        try:
            eq = float(cycle_equity) if cycle_equity is not None else float(get_account_equity(api))
        except Exception:
            eq = float("nan")
        try:
            px = float(bars_df["Close"].iloc[-1]) if not bars_df.empty else float(get_last_price(api, symbol))
        except Exception:
            px = float("nan")

        _NO_POS_CYCLE_COUNT[symbol] = 0
        log_trade_symbol(
            symbol,
            bars_df.index[-1] if not bars_df.empty else pd.NaT,
            signal=0,
            raw_action=0.0,
            weight=0.0,
            confidence=0.0,
            price=px,
            equity=eq,
            dry_run=DRY_RUN,
            note="reentry_cooldown",
        )
        return

    min_rows_needed = max(20, int(shape[0]) if (shape and len(shape) == 2) else 60)
    try:
        obs, obs_ts = prepare_observation_from_bars(
            bars_df,
            features_hint=features_hint,
            min_required_rows=min_rows_needed,
            expected_shape=shape,
            symbol=symbol,
        )
    except Exception as e:
        logging.info("[%s] Could not prepare observation (%s); skipping.", symbol, e)
        try:
            eq = float(cycle_equity) if cycle_equity is not None else float(get_account_equity(api))
        except Exception:
            eq = float("nan")
        try:
            px = float(bars_df["Close"].iloc[-1]) if not bars_df.empty else float(get_last_price(api, symbol))
        except Exception:
            px = float("nan")
        log_trade_symbol(
            symbol,
            bars_df.index[-1] if not bars_df.empty else pd.NaT,
            signal=0,
            raw_action=0.0,
            weight=0.0,
            confidence=0.0,
            price=px,
            equity=eq,
            dry_run=DRY_RUN,
            note="obs_build_failed",
        )
        return

    _obs_shape = getattr(obs, "shape", None)
    _vecnorm_str = (
        f"{type(vecnorm).__name__}(training={getattr(vecnorm,'training',None)}, norm_reward={getattr(vecnorm,'norm_reward',None)})"
    ) if vecnorm is not None else "None"
    _now_ts = utc_ts(now_utc())
    _age = _now_ts - int(obs_ts)
    logging.info("[%s] obs_shape=%s | exp_shape=%s | age=%ss | vecnorm=%s",
                 symbol, _obs_shape, shape, _age, _vecnorm_str)
    #shape check + predict heartbeat
    exp = expected_obs_shape(model, vecnorm)
    if exp is not None and hasattr(obs, "shape"):
        logging.info("[%s] shape_check obs=%s expected=%s", symbol, getattr(obs, "shape", None), exp)
    logging.info("[%s]  obs built. Calling model.predict()", symbol)

    if _now_ts - obs_ts >= STALE_MAX_SEC:
        logging.info(f"[{symbol}] Observation stale (age={_now_ts-obs_ts}s ≥ {STALE_MAX_SEC}s); skipping.")
        try:
            eq = get_account_equity(api)
            px = float(bars_df["Close"].iloc[-1]) if not bars_df.empty else get_last_price(api, symbol)
        except Exception:
            eq, px = float("nan"), float("nan")
        log_trade_symbol(symbol, bars_df.index[-1] if not bars_df.empty else pd.NaT,
                         0, 0.0, 0.0, 0.0, px, eq, DRY_RUN, note="skip_stale")
        return

    if check_tp_sl_and_maybe_flatten(api, symbol):
        return

    # ---- DIAGNOSTIC: obs stats (raw vs normalized) ----
    x = np.asarray(obs, dtype=np.float32)
    x2 = x.copy()
    if vecnorm is not None and getattr(vecnorm, "obs_rms", None) is not None:
        try:
            x2 = vecnorm.normalize_obs(x2)
        except Exception:
            x2 = vecnorm.normalize_obs(np.expand_dims(x2, axis=0))[0]

    logging.info(
        "[%s] obs stats raw: mean=%.4f std=%.4f | normed: mean=%.4f std=%.4f",
        symbol, float(x.mean()), float(x.std()),
        float(np.asarray(x2).mean()), float(np.asarray(x2).std())
    )

    # IMPORTANT: infer_target_weight() already applies vecnorm normalization internally
    target_w, conf, raw = infer_target_weight(model, vecnorm, obs)
    logging.info("[%s] predict() ok → raw=%.4f target_w=%.4f conf=%.3f", symbol, raw, target_w, conf)
    eq = float(cycle_equity) if cycle_equity is not None else get_account_equity(api)
    px = float(bars_df["Close"].iloc[-1]) if not bars_df.empty else get_last_price(api, symbol)
    have = get_position_qty(api, symbol)

    #FORCE_FIRST_BUY: one-time seed entry if no position exists
    if FORCE_FIRST_BUY and (have == 0) and (not _FORCED_FIRST_BUY_DONE.get(symbol, False)):
        if ensure_market_open(api):
            if begin_order_event(symbol, _SEED_COOLDOWN_SEC):
                tradable, fractionable, shortable = _asset_flags(symbol)
                seed_notional = round_to_cents(float(globals().get("REBALANCE_MIN_NOTIONAL", 5.00)))

                # seed direction = model direction (not always BUY)
                if target_w < 0:
                    # seed short only if allowed & shortable
                    if not ALLOW_SHORTS or not shortable:
                        log_trade_symbol(
                            symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN,
                            note="force_first_seed_short_blocked"
                        )
                        _FORCED_FIRST_BUY_DONE[symbol] = True
                        return

                    # short seed uses 1 share (fractional short not supported here safely)
                    o = market_order_to_qty(api, symbol, side="sell", qty=1)

                else:
                    # long seed
                    if USE_FRACTIONALS and fractionable:
                        o = market_order(api, symbol, side="buy", notional=seed_notional)
                    else:
                        o = market_order_to_qty(api, symbol, side="buy", qty=1)

                stamp_order_event(symbol)
                _FORCED_FIRST_BUY_DONE[symbol] = True

                log_trade_symbol(
                    symbol=symbol,
                    bar_time=bars_df.index[-1],
                    signal=1,
                    raw_action=raw,
                    weight=target_w,
                    confidence=conf,
                    price=px,
                    equity=eq,
                    dry_run=DRY_RUN,
                    note="force_first_seed_directional",
                    **_order_info(o),
                )
                return

        logging.info(f"[{symbol}] raw={raw:.4f} conf={conf:.3f} → target_w={target_w:.4f} px=${px:.2f} eq=${eq:,.2f} have={have}")

        if os.getenv("DEBUG_FORCE_SEED_IF_IDLE", "0").lower() in ("1", "true", "yes"):
            if have != 0:
                _NO_POS_CYCLE_COUNT[symbol] = 0
            else:
                _NO_POS_CYCLE_COUNT[symbol] = _NO_POS_CYCLE_COUNT.get(symbol, 0) + 1

            idle_cycles = int(os.getenv("DEBUG_SEED_IDLE_CYCLES", "10"))
            if have == 0 and _NO_POS_CYCLE_COUNT[symbol] >= idle_cycles and ensure_market_open(api):
                tradable, fractionable, _ = _asset_flags(symbol)
                if not tradable:
                    log_trade_symbol(symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN, note="not_tradable_seed")
                    return
                seed_amt = round_to_cents(REBALANCE_MIN_NOTIONAL)
                if USE_FRACTIONALS and fractionable:
                    market_order(api, symbol, side="buy", notional=seed_amt)
                else:
                    market_order_to_qty(api, symbol, side="buy", qty=1)
                log_trade_symbol(symbol, bars_df.index[-1], 1, raw, target_w, conf, px, eq, DRY_RUN, note="debug_force_seed")
                return

    RAW_POS_MIN_LOCAL = float(globals().get("RAW_POS_MIN", 0.0))
    if target_w > 0 and raw < RAW_POS_MIN_LOCAL:
        logging.info(f"[{symbol}] Raw {raw:.4f} < RAW_POS_MIN {RAW_POS_MIN_LOCAL:.4f}; no action.")
        log_trade_symbol(symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN, note="raw_gate_long")
        return

    RAW_NEG_GATE = float(globals().get("RAW_NEG_MAX", 0.0))
    if target_w < 0 and abs(raw) < RAW_NEG_GATE:
        logging.info(f"[{symbol}] |raw| {abs(raw):.4f} < RAW_NEG_GATE {RAW_NEG_GATE:.4f}; no action.")
        log_trade_symbol(symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN, note="raw_gate_short")
        return

    pos = get_position(api, symbol)
    if abs(target_w) <= EXIT_WEIGHT_MAX and pos:
        logging.info(f"[{symbol}] Model near-flat (≤{EXIT_WEIGHT_MAX:.3f}); flattening.")
        flatten_symbol(api, symbol)
        log_trade_symbol(symbol, bars_df.index[-1], int(target_w > 0), raw, target_w, conf, px, eq, DRY_RUN, note="flatten")
        return

    if conf < ENTER_CONF_MIN and abs(target_w) <= EXIT_WEIGHT_MAX:
        logging.info(f"[{symbol}] Below conf/near-flat gates; no action.")
        log_trade_symbol(symbol, bars_df.index[-1], int(target_w > 0), raw, target_w, conf, px, eq, DRY_RUN, note="no_action")
        return

    wants_trade = (abs(target_w) >= ENTER_WEIGHT_MIN and conf >= ENTER_CONF_MIN)
    has_pos = (have != 0)

    if wants_trade:
        event_gap = _SEED_COOLDOWN_SEC if (SEED_FIRST_SHARE and not has_pos) else 30
        if not begin_order_event(symbol, event_gap):
            note = "order_event_cooldown_seed" if (SEED_FIRST_SHARE and not has_pos) else "order_event_cooldown_rebalance"
            log_trade_symbol(symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN, note=note)
            return

        tradable, fractionable, _shortable = _asset_flags(symbol)
        if not tradable:
            log_trade_symbol(symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN, note="not_tradable")
            return

        seeded = False
        if SEED_FIRST_SHARE and not has_pos:
            seed_notional = round_to_cents(REBALANCE_MIN_NOTIONAL)
            side = "buy" if target_w > 0 else "sell"

            if side == "sell":
                ok, why = _can_seed_short(api, symbol)
                if not ok:
                    log_trade_symbol(symbol, bars_df.index[-1], 0, raw, target_w, conf, px, eq, DRY_RUN, note=why)
                    return
                _ = market_order_to_qty(api, symbol, side="sell", qty=1)
                seeded = True
            else:
                if USE_FRACTIONALS and fractionable:
                    _ = market_order(api, symbol, side="buy", notional=seed_notional)
                else:
                    _ = market_order_to_qty(api, symbol, side="buy", qty=1)
                seeded = True

        order_info = rebalance_to_weight(api, symbol, eq, target_w) or dict(NO_ORDER)

        if seeded or int(order_info.get("order_submitted", 0)) == 1 or DRY_RUN:
            stamp_order_event(symbol)

        note = "seed+rebalance" if seeded else "rebalance_only"
        log_trade_symbol(
            symbol, bars_df.index[-1],
            int(target_w > 0),
            raw, target_w, conf, px, eq, DRY_RUN,
            note=note,
            order_submitted=order_info.get("order_submitted", 0),
            order_id=order_info.get("order_id", ""),
            order_status=order_info.get("order_status", ""),
            filled_qty=order_info.get("filled_qty", ""),
        )
        return

# ============================================================
# Live runner
# ============================================================

def run_live(tickers: List[str], api: tradeapi.REST):
    def minutes_to_close(api: tradeapi.REST) -> Optional[int]:
        clk = api.get_clock()
        if getattr(clk, "is_open", False):
            close = pd.to_datetime(clk.next_close, utc=True)
            return int(max(0, (close - now_utc()).total_seconds() // 60))
        return None

    api_local = api
    per_ticker: Dict[str, Tuple[PPO, Optional[VecNormalize], Optional[dict]]] = {}
    best = (globals().get("BEST_WINDOW_ENV") or None)

    for t in tickers:
        try:
            picks = pick_artifacts_for_ticker(t, os.getenv("ARTIFACTS_DIR", str(ARTIFACTS_DIR)), best_window=best)
            model = load_ppo_model(picks["model"])
            vecnorm = load_vecnormalize(picks.get("vecnorm"))
            if vecnorm is not None and hasattr(vecnorm, "training"):
                vecnorm.training = False
            if vecnorm is not None and hasattr(vecnorm, "norm_reward"):
                vecnorm.norm_reward = False
            feats = load_features(picks.get("features"))
            per_ticker[t] = (model, vecnorm, feats)
            logging.info("[%s] Artifacts loaded and ready.", t)
        except Exception as e:
            logging.exception("[%s] Failed to load artifacts: %s", t, e)

    if not per_ticker:
        raise RuntimeError("No models loaded for any ticker. Check artifacts directory and names.")

    loaded_syms = list(per_ticker.keys())
    logging.info("Starting live execution for (loaded): %s", loaded_syms)

    global _last_kill_ts

    def run_symbol_step_safe(
        symbol: str,
        model: PPO,
        vecnorm: Optional[VecNormalize],
        feat_hint: Optional[dict],
        cycle_equity: float,
        timeout_sec: int = 15,
    ):
        return _call_with_timeout(
            run_live_once_for_symbol,
            timeout_sec,
            api_local,
            symbol,
            model,
            vecnorm,
            feat_hint,
            cycle_equity,
        )

    cycle = 0
    last_plot_ts = 0
    flattened_today = False

    logging.info("Starting live trading loop")
    try:
        while True:
            if not ensure_market_open(api_local):
                flattened_today = False
                globals()["SESSION_OPEN_EQUITY"] = None
                _sleep_until_open(api_local)
                continue

            if globals().get("SESSION_OPEN_EQUITY") is None:
                try:
                    globals()["SESSION_OPEN_EQUITY"] = float(api_local.get_account().equity)
                    logging.info(
                        "Session open equity anchor set: %.2f",
                        globals()["SESSION_OPEN_EQUITY"],
                    )
                except Exception as e:
                    logging.debug("Could not set SESSION_OPEN_EQUITY: %s", e)

            t_cycle_start = time.perf_counter()

            try:
                cycle_equity = float(api_local.get_account().equity)
            except Exception as e:
                logging.warning("Could not fetch equity: %s", e)
                cycle_equity = float("nan")

            print(
                f"[HEARTBEAT] {utcnow_iso()} cycle={cycle} equity={cycle_equity:,.2f}",
                flush=True,
            )

            for sym, (model, vecnorm, feat_hint) in per_ticker.items():
                t_sym_start = time.perf_counter()
                try:
                    run_symbol_step_safe(
                        sym,
                        model,
                        vecnorm,
                        feat_hint,
                        cycle_equity,
                        timeout_sec=15,
                    )
                except Exception as e:
                    logging.warning("[%s] symbol step timeout/fail: %s", sym, e)
                finally:
                    logging.info(
                        "[TIMER] %s symbol work: %.3fs",
                        sym,
                        time.perf_counter() - t_sym_start,
                    )

            maybe_log_equity_snapshot(
                api_in=api_local,
                reason=("trade" if globals().get("_TRADE_EVENT_FLAG", False) else "cycle"),
            )

            # Kill-switch
            try:
                anchor = globals().get("SESSION_OPEN_EQUITY", None)
                if anchor is not None:
                    eq_now = float(api_local.get_account().equity)
                    dd = (eq_now / max(1e-9, float(anchor))) - 1.0
                    max_dd = float(
                        os.getenv(
                            "MAX_DAILY_DRAWDOWN_PCT",
                            globals().get("MAX_DAILY_DRAWDOWN_PCT", 0.05),
                        )
                    )
                    if dd <= -abs(max_dd):
                        if time.time() - _last_kill_ts > 60:
                            for s in per_ticker.keys():
                                flatten_symbol(api_local, s)
                            logging.warning(
                                "KILL-SWITCH: daily drawdown %.2f%% reached. Flattening & pausing.",
                                100.0 * dd,
                            )
                            _last_kill_ts = time.time()

                            cooldown_min = int(
                                os.getenv(
                                    "KILL_SWITCH_COOLDOWN_MIN",
                                    str(globals().get("KILL_SWITCH_COOLDOWN_MIN", 30)),
                                )
                            )
                            if not DRY_RUN:
                                time.sleep(60 * cooldown_min)
                            continue
            except Exception as e:
                logging.debug("kill-switch check failed: %s", e)

            # Flatten into close
            m2c = minutes_to_close(api_local)
            if FLATTEN_INTO_CLOSE and not flattened_today and m2c is not None and m2c <= 5:
                for s in per_ticker.keys():
                    flatten_symbol(api_local, s)
                    _REENTRY_BLOCK_UNTIL[s] = time.time() + REENTRY_COOLDOWN_SEC
                logging.info("Flattened all positions into the close.")
                maybe_log_equity_snapshot(api_in=api_local, reason="close")
                flattened_today = True

                if bool(globals().get("EXIT_AFTER_CLOSE", False)):
                    logging.info("EXIT_AFTER_CLOSE=True — exiting live loop after close flatten.")
                    break

            cycle += 1

            now_ts = time.time()
            if now_ts - last_plot_ts >= 900:
                try:
                    plot_equity_curve(from_equity_csv=True)
                    df = pd.read_csv(EQUITY_LOG_CSV, parse_dates=["datetime_utc"])
                    m = compute_performance_metrics(df)
                    logging.info(
                        "Perf: cum_return=%.2f%% | sharpe=%.2f | maxDD=%.2f%%",
                        100 * m["cum_return"],
                        m["sharpe"],
                        100 * m["max_drawdown"],
                    )
                except Exception as e:
                    logging.warning("Plot/metrics failed: %s", e)
                last_plot_ts = now_ts

            logging.info(
                "[TIMER] full-cycle active time: %.3fs (cooldown=%d min)",
                time.perf_counter() - t_cycle_start,
                COOLDOWN_MIN,
            )

            if (cycle % 12) == 0:
                gc.collect()

            _sleep_to_next_minute_block(COOLDOWN_MIN)

    except KeyboardInterrupt:
        logging.info("KeyboardInterrupt: stopping live loop.")
    except Exception as e:
        logging.exception("Live loop exception: %s", e)
        try:
            log_equity_snapshot(api_in=api_local)
        except Exception:
            pass
    finally:
        global _TIMEOUT_EXEC
        try:
            _TIMEOUT_EXEC.shutdown(wait=False, cancel_futures=True)
        except Exception:
            pass

        _TIMEOUT_EXEC = ThreadPoolExecutor(max_workers=8)
        logging.info("Timeout executor reset.")

        try:
            if FORCE_FLATTEN_ON_EXIT:
                flatten_all_positions(api_local)
        except Exception as e:
            logging.warning("Flatten-on-exit skipped: %s", e)

        try:
            maybe_log_equity_snapshot(api_in=api_local, reason="finalize")
            plot_equity_curve(from_equity_csv=True)
        except Exception as e:
            logging.warning("Finalization failed: %s", e)

        logging.info("Live loop exited cleanly.")

# ============================================================
# Diagnostic runner (optional)
# ============================================================

def ticker_diagnostic(ticker: str, artifacts_dir: str, lookback_bars: int = 200):
    logging.info("[diagnostic] starting for %s", ticker)
    api_local = init_alpaca()

    art = pick_artifacts_for_ticker(ticker, artifacts_dir)
    model_path = art.get("model")
    vec_path = art.get("vecnorm")
    feats_path = art.get("features")

    if model_path is None:
        logging.error("[%s] no model found; aborting diagnostic", ticker)
        return None

    model = load_ppo_model(model_path)
    vecnorm = load_vecnormalize(vec_path)
    if vecnorm is not None and hasattr(vecnorm, "training"):
        vecnorm.training = False
    if vecnorm is not None and hasattr(vecnorm, "norm_reward"):
        vecnorm.norm_reward = False

    feats = load_features(feats_path) if feats_path else None

    bars_df = get_recent_bars(api_local, ticker, limit=lookback_bars, timeframe=LIVE_TIMEFRAME)
    if bars_df is None or bars_df.empty:
        logging.error("[%s] no bars returned", ticker)
        return None

    shape = expected_obs_shape(model, vecnorm)
    obs, obs_ts = prepare_observation_from_bars(bars_df, features_hint=feats, expected_shape=shape, symbol=ticker)

    if vecnorm is not None and getattr(vecnorm, "obs_rms", None) is not None:
        try:
            obs = vecnorm.normalize_obs(obs)
        except Exception:
            obs = vecnorm.normalize_obs(np.expand_dims(obs, axis=0))[0]

    action, _ = model.predict(obs, deterministic=True)
    logging.info("[diagnostic %s] action=%s | obs_ts=%s | bars=%d | timeframe=%s",
                 ticker, action, obs_ts, len(bars_df), LIVE_TIMEFRAME)
    return {"ticker": ticker, "action": action, "obs_ts": obs_ts, "bars": len(bars_df), "timeframe": str(LIVE_TIMEFRAME)}

# ============================================================
# Config banner + logging setup after paths
# ============================================================

def log_config_banner():
    try:
        artifacts_list = sorted(p.name for p in ARTIFACTS_DIR.iterdir()) if ARTIFACTS_DIR.exists() else []
    except Exception:
        artifacts_list = []

    logging.info("EXIT_AFTER_CLOSE      : %s", os.getenv("EXIT_AFTER_CLOSE", "0"))
    logging.info("FORCE_FIRST_BUY       : %s", FORCE_FIRST_BUY)
    logging.info("FORCE_FLATTEN_ON_EXIT : %s", FORCE_FLATTEN_ON_EXIT)
    logging.info("CONFIG")
    logging.info("Project root          : %s", PROJECT_ROOT)
    logging.info("ARTIFACTS_DIR         : %s", ARTIFACTS_DIR)
    logging.info("RESULTS_DIR           : %s", RESULTS_DIR)
    logging.info("Tickers               : %s", TICKERS)
    logging.info("API base              : %s", BASE_URL)
    logging.info("AUTO_RUN_LIVE         : %s", os.getenv("AUTO_RUN_LIVE", ""))
    logging.info("INF_DETERMINISTIC     : %s", INF_DETERMINISTIC)
    logging.info("ALLOW_SHORTS          : %s", ALLOW_SHORTS)
    logging.info("FLATTEN_INTO_CLOSE    : %s", FLATTEN_INTO_CLOSE)
    logging.info("REENTRY_COOLDOWN_SEC  : %s", os.getenv("REENTRY_COOLDOWN_SEC", str(REENTRY_COOLDOWN_SEC)))
    logging.info("DRY_RUN=%s | BARS_FEED=%s | USE_FRACTIONALS=%s | COOLDOWN_MIN=%s | STALE_MAX_SEC=%s",
                 DRY_RUN, BARS_FEED, USE_FRACTIONALS, COOLDOWN_MIN, STALE_MAX_SEC)

    logging.info("DEBUG_FORCE_SEED_IF_IDLE=%s | DEBUG_SEED_IDLE_CYCLES=%s",
                 os.getenv("DEBUG_FORCE_SEED_IF_IDLE", "0"), os.getenv("DEBUG_SEED_IDLE_CYCLES", "10"))

    logging.info("PH_TIMEOUT_SEC        : %s", os.getenv("PH_TIMEOUT_SEC", "8"))
    logging.info("DATA_TIMEFRAME        : %s (model bars)", os.getenv("DATA_TIMEFRAME", "1H"))
    logging.info("EQUITY_TIMEFRAME      : %s (equity reporting)", os.getenv("EQUITY_TIMEFRAME", "5Min"))

    logging.info("MAX_DD_PCT: %.3f | KILL_SWITCH_COOLDOWN_MIN: %s",
                 float(globals().get("MAX_DAILY_DRAWDOWN_PCT", 0.05)),
                 os.getenv("KILL_SWITCH_COOLDOWN_MIN", str(globals().get("KILL_SWITCH_COOLDOWN_MIN", 30))))

    logging.info("WEIGHT_CAP: %.3f | SIZING_MODE: %s | ENTER_CONF_MIN: %.3f | ENTER_WEIGHT_MIN: %.3f | EXIT_WEIGHT_MAX: %.3f | REBALANCE_MIN_NOTIONAL: %.2f",
                 WEIGHT_CAP, SIZING_MODE, ENTER_CONF_MIN, ENTER_WEIGHT_MIN, EXIT_WEIGHT_MAX, REBALANCE_MIN_NOTIONAL)

    logging.info("TAKE_PROFIT_PCT: %.3f | STOP_LOSS_PCT: %.3f | BEST_WINDOW_ENV: %s",
                 TAKE_PROFIT_PCT, STOP_LOSS_PCT, (BEST_WINDOW_ENV or ""))

    logging.info("DELTA_WEIGHT_MIN: %.3f | RAW_POS_MIN: %.3f | RAW_NEG_MAX: %.3f",
                 float(globals().get("DELTA_WEIGHT_MIN", 0.0)),
                 float(globals().get("RAW_POS_MIN", 0.0)),
                 float(globals().get("RAW_NEG_MAX", 0.0)))

    if artifacts_list:
        logging.info("Artifacts present (%d): %s", len(artifacts_list), ", ".join(artifacts_list))

def setup_logging_after_paths():
    warnings.filterwarnings("default")
    level = getattr(logging, os.getenv("LOG_LEVEL", "INFO").upper(), logging.INFO)

    root = logging.getLogger()
    root.handlers.clear()
    root.setLevel(level)

    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")

    sh = logging.StreamHandler(sys.stdout)
    sh.setLevel(level)
    sh.setFormatter(fmt)
    root.addHandler(sh)

    log_path = RESULTS_DIR / "live_loop.log"
    fh = logging.FileHandler(log_path)
    fh.setLevel(level)
    fh.setFormatter(fmt)
    root.addHandler(fh)

    try:
        sys.stdout.reconfigure(line_buffering=True)
    except Exception:
        pass

# ============================================================
# Main
# ============================================================

if __name__ == "__main__":
    if IN_COLAB:
        upload_env_and_artifacts_in_colab()
        normalize_artifacts()
        load_dotenv(dotenv_path=PROJECT_ROOT / ".env", override=True)
        REENTRY_COOLDOWN_SEC = int(os.getenv("REENTRY_COOLDOWN_SEC", "300"))

    cfg = configure_knobs(overrides={
        # data freshness
        "BARS_FEED": "",
        "STALE_MAX_SEC": 4200,

        # sizing & threshold shaping
        "SIZING_MODE": "linear",
        "CONF_FLOOR": 0.00,
        "WEIGHT_CAP": 0.40,

        # entry/exit sensitivity
        "ENTER_CONF_MIN": 0.02,
        "ENTER_WEIGHT_MIN": 0.002,
        "EXIT_WEIGHT_MAX": 0.001,
        "DELTA_WEIGHT_MIN": 0.0005,
        "REBALANCE_MIN_NOTIONAL": 5.00,
        # --- execution ---
        "FORCE_FIRST_BUY": True,

        # posture
        "ALLOW_SHORTS": True,
        "COOLDOWN_MIN": 10,

        # raw-action gates
        "RAW_POS_MIN": 0.00,
        "RAW_NEG_MAX": 0.00,

        # risk
        "TAKE_PROFIT_PCT": 0.05,
        "STOP_LOSS_PCT": 0.02,

        # logging cadence
        "EQUITY_LOG_THROTTLE_SEC": 300,
        "SKIP_EQUITY_WHEN_DRY_RUN": False,

        # kill-switch
        "MAX_DAILY_DRAWDOWN_PCT": 0.05,
    })
    globals()["cfg"] = cfg

    setup_logging_after_paths()

    # Live data timeframe (match PPO training granularity)
    LIVE_TIMEFRAME = _TF_MAP.get(cfg.DATA_TIMEFRAME.strip().lower(), TimeFrame.Hour)

    TRAIN_TIMEFRAME = os.getenv("TRAIN_TIMEFRAME", "1H").strip().lower()
    if cfg.DATA_TIMEFRAME.strip().lower() != TRAIN_TIMEFRAME:
        logging.warning(
            "⚠️ Timeframe mismatch: trained=%s live=%s. Only change DATA_TIMEFRAME if you retrained the model.",
            TRAIN_TIMEFRAME, cfg.DATA_TIMEFRAME
        )

    if cfg.AUTO_RUN_LIVE:
        assert "paper-api" in BASE_URL.lower()

    log_config_banner()
    logging.info("DATA_TIMEFRAME=%s -> LIVE_TIMEFRAME=%s", cfg.DATA_TIMEFRAME, LIVE_TIMEFRAME)

    # Save run config snapshot
    try:
        cfg_path = RESULTS_DIR / "run_config.json"
        payload = {
            "time": utcnow_iso(),
            "tickers": TICKERS,
            "dry_run": DRY_RUN,
            "bars_feed": BARS_FEED,
            "weight_cap": WEIGHT_CAP,
            "enter_conf_min": ENTER_CONF_MIN,
            "enter_weight_min": ENTER_WEIGHT_MIN,
            "exit_weight_max": EXIT_WEIGHT_MAX,
            "rebalance_min_notional": REBALANCE_MIN_NOTIONAL,
            "delta_weight_min": DELTA_WEIGHT_MIN,
            "tp": TAKE_PROFIT_PCT,
            "sl": STOP_LOSS_PCT,
            "allow_shorts": ALLOW_SHORTS,
        }
        tmp = cfg_path.with_suffix(".tmp")
        tmp.write_text(json.dumps(payload, indent=2))
        tmp.replace(cfg_path)
    except Exception as e:
        logging.warning("Could not write run_config.json: %s", e)

    # Paper safety
    assert "paper-api" in BASE_URL.lower(), f"Refusing to trade: BASE_URL is not paper ({BASE_URL})"

    # Single init
    api = init_alpaca()
    acct = api.get_account()

    logging.info("shorting_enabled=%s", getattr(acct, "shorting_enabled", None))

    # Optional sanity
    assert not bool(getattr(acct, "trading_blocked", False)), f"Trading is blocked on this account: {getattr(acct,'status','')}"

    logging.info("Account status: %s | equity=%s | cash=%s", acct.status, acct.equity, acct.cash)
    write_account_info_to_run_config(api)

    if cfg.AUTO_RUN_LIVE:
        run_live(TICKERS, api)
    else:
        logging.info("AUTO_RUN_LIVE disabled; live loop not started.")

Mounted at /content/drive
Upload your .env (or Alpaca_keys.env.txt). Cancel if already on Drive.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Saving Alpaca_keys.env.txt to Alpaca_keys.env.txt
Saved env → /content/drive/MyDrive/AlpacaPaper/.env
Upload your artifacts (ppo_*_model.zip, *_vecnorm*.pkl, *_features*.json or .txt).


Saving ppo_GE_window1_features.json to ppo_GE_window1_features.json
Saving ppo_GE_window1_model_info.json to ppo_GE_window1_model_info.json
Saving ppo_GE_window1_model.zip to ppo_GE_window1_model.zip
Saving ppo_GE_window1_probability_config.json to ppo_GE_window1_probability_config.json
Saving ppo_GE_window1_vecnorm.pkl to ppo_GE_window1_vecnorm.pkl
Saving ppo_UNH_window3_features.json to ppo_UNH_window3_features.json
Saving ppo_UNH_window3_model_info.json to ppo_UNH_window3_model_info.json
Saving ppo_UNH_window3_model.zip to ppo_UNH_window3_model.zip
Saving ppo_UNH_window3_probability_config.json to ppo_UNH_window3_probability_config.json
Saving ppo_UNH_window3_vecnorm.pkl to ppo_UNH_window3_vecnorm.pkl
Artifacts now in: ['ppo_CVX_window1_features.json', 'ppo_CVX_window1_model.zip', 'ppo_CVX_window1_model_info.json', 'ppo_CVX_window1_probability_config.json', 'ppo_CVX_window1_vecnorm.pkl', 'ppo_GE_window1_features.json', 'ppo_GE_window1_model.zip', 'ppo_GE_window1_model_info.json', 'p

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


...

2026-02-11 20:50:01,694 | INFO | [GE] raw=0.1911 conf=0.191 → target_w=0.0764 px=$314.83 eq=$99,357.98 have=24.155996526
2026-02-11 20:50:02,035 | INFO | [TIMER] GE symbol work: 0.684s
2026-02-11 20:50:02,432 | INFO | [TIMER] full-cycle active time: 2.133s (cooldown=10 min)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


2026-02-11 21:00:00,382 | INFO | Market closed. Sleeping 62999s until next open.
2026-02-11 21:00:08,775 | INFO | KeyboardInterrupt: stopping live loop.
2026-02-11 21:00:08,778 | INFO | Timeout executor reset.


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7ff8da831940>
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Saved equity curve → /content/drive/MyDrive/AlpacaPaper/results/2026-02-11/equity_curve.png
Updated latest copy → /content/drive/MyDrive/AlpacaPaper/results/latest/equity_curve.png
2026-02-11 21:00:09,368 | INFO | Live loop exited cleanly.


In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from alpaca_trade_api.rest import TimeFrame


RESULTS_DIR = Path(globals().get("RESULTS_DIR", os.getenv("RESULTS_DIR", ".")))
LATEST_DIR  = Path(globals().get("LATEST_DIR",  os.getenv("LATEST_DIR",  str(RESULTS_DIR))))

eq_candidates = [
    globals().get("EQUITY_LOG_CSV"),
    globals().get("EQUITY_LOG_LATEST"),
    RESULTS_DIR / "equity_log.csv",
    LATEST_DIR / "equity_log.csv",
]

def _first_existing(paths):
    for p in paths:
        if p:
            p = Path(p)
            if p.exists() and p.is_file():
                return p
    return None

eq_path = _first_existing(eq_candidates)
if eq_path is None:
    all_eq = list(RESULTS_DIR.glob("equity_log*.csv")) + list(LATEST_DIR.glob("equity_log*.csv"))
    eq_path = max(all_eq, key=lambda p: p.stat().st_mtime, default=None)

if eq_path and eq_path.exists():
    print(f"[equity source] {eq_path}")
    try:
        eq = pd.read_csv(eq_path, parse_dates=["datetime_utc"]).sort_values("datetime_utc")
        if not eq.empty:
            r = eq["equity"].pct_change().dropna()
            sharpe_h = (r.mean() / (r.std() + 1e-12)) * np.sqrt(252 * 6.5) if len(r) else float("nan")
            print(
                f"\nEquity summary — last: ${eq['equity'].iloc[-1]:,.2f} | "
                f"n={len(eq)} pts | Sharpe(h): {sharpe_h:.2f} | src={eq_path}"
            )
        else:
            print(f"No rows in equity log: {eq_path}")
    except Exception as e:
        print(f"Could not summarize equity ({eq_path}): {e}")
else:
    print("No equity_log*.csv found in RESULTS_DIR/LATEST_DIR.")

def _resolve_tickers():
    g = globals().get("TICKERS", None)

    # Base tickers from globals or env
    if isinstance(g, (list, tuple, set)):
        base = [str(x).upper() for x in g]
    else:
        env_val = os.getenv("TICKERS", (g if isinstance(g, str) else ""))
        base = [t.strip().upper() for t in str(env_val).split(",") if t.strip()]

    # Also include symbols with existing logs on disk
    discovered = [
        p.stem.replace("trade_log_", "").upper()
        for p in list(RESULTS_DIR.glob("trade_log_*.csv")) + list(LATEST_DIR.glob("trade_log_*.csv"))
    ]

    ticks = sorted(set(base) | set(discovered))
    return ticks if ticks else ["UNH", "GE"]

tickers_to_report = _resolve_tickers()
print("Tickers to report:", tickers_to_report)

print("\nTrade Summary:")
for ticker in tickers_to_report:
    trade_candidates = [
        RESULTS_DIR / f"trade_log_{ticker}.csv",
        LATEST_DIR / f"trade_log_{ticker}.csv",
    ]
    log_path = _first_existing(trade_candidates)
    if not log_path:
        #Tolerate Drive duplicates like "trade_log_XYZ (1).csv"
        any_logs = list(RESULTS_DIR.glob(f"trade_log_{ticker}*.csv")) + \
                   list(LATEST_DIR.glob(f"trade_log_{ticker}*.csv"))
        log_path = max(any_logs, key=lambda p: p.stat().st_mtime, default=None)

    if not log_path or not log_path.exists():
        print(f"{ticker}: no trades logged yet.")
        continue

    try:
        df = pd.read_csv(
            log_path,
            on_bad_lines="skip",
            engine="python",
            parse_dates=["log_time", "bar_time"],
        )
        key = "signal" if "signal" in df.columns else ("action" if "action" in df.columns else None)
        if key:
            counts = df[key].value_counts(dropna=False).to_dict()
            print(f"{ticker}: {counts} | src={log_path.name}")
        else:
            print(f"{ticker}: log present but missing 'signal'/'action' columns. src={log_path.name}")

        if "confidence" in df.columns and df["confidence"].notna().any():
            plt.figure(figsize=(8, 3.5))
            df["confidence"].dropna().plot(kind="hist", bins=10, edgecolor="black")
            plt.title(f"{ticker} - Confidence Distribution")
            plt.xlabel("confidence")
            plt.tight_layout()
            plt.show()

        for col in ["weight", "raw_action"]:
            if col in df.columns and df[col].notna().any():
                s = df[col].dropna()
                print(
                    f"{ticker} {col}: mean={s.mean():.3f}, std={s.std():.3f}, "
                    f"min={s.min():.3f}, max={s.max():.3f}"
                )
    except Exception as e:
        print(f"{ticker}: could not summarize trades ({log_path}): {e}")

try:
    if "api" not in globals():
        api = init_alpaca()
    positions = api.list_positions()
    total_market_value = 0.0
    print("\nPosition Summary:")
    for p in positions:
        mv = float(p.market_value)
        total_market_value += mv
        print(f"  {p.symbol}: {p.qty} shares @ ${float(p.current_price):.2f} | Value: ${mv:,.2f}")
    print(f"\nTotal Market Value: ${total_market_value:,.2f}")
except Exception as e:
    print(f"Could not summarize positions: {e}")

from datetime import datetime, timedelta, timezone

def count_filled_orders_since(api, symbol: str, days: int = 14) -> int:
    after = (datetime.now(timezone.utc) - timedelta(days=days)).isoformat()
    orders = api.list_orders(status="all", after=after, nested=True)
    return sum(1 for o in orders if o.symbol == symbol and o.status in ("filled", "partially_filled"))

try:
    api_chk = api if "api" in globals() else init_alpaca()
    for sym in tickers_to_report:
        n = count_filled_orders_since(api_chk, sym, days=14)
        print(f"{sym}: {n} filled trades in last 14 days")
except Exception as e:
    print(f"Could not fetch filled orders: {e}")


[equity source] /content/drive/MyDrive/AlpacaPaper/results/2026-02-11/equity_log.csv

Equity summary — last: $99,331.40 | n=8 pts | Sharpe(h): -2.87 | src=/content/drive/MyDrive/AlpacaPaper/results/2026-02-11/equity_log.csv
Tickers to report: ['GE', 'MASTER', 'UNH']

Trade Summary:
GE: {'LONG': 7} | src=trade_log_GE.csv


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


GE weight: mean=0.086, std=0.012, min=0.076, max=0.099
GE raw_action: mean=0.215, std=0.030, min=0.191, max=0.247
MASTER: no trades logged yet.
UNH: {'SHORT': 6, 'FLAT': 1} | src=trade_log_UNH.csv
UNH weight: mean=-0.157, std=0.069, min=-0.183, max=0.000
UNH raw_action: mean=-0.392, std=0.173, min=-0.458, max=0.000

Position Summary:
  GE: 24.155996526 shares @ $313.73 | Value: $7,578.46

Total Market Value: $7,578.46


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


GE: 11 filled trades in last 14 days
MASTER: 0 filled trades in last 14 days
UNH: 6 filled trades in last 14 days


In [ ]:
#--- Export locally & download to your computer (Colab) ---
from pathlib import Path
from datetime import datetime, timezone
from google.colab import files   #<-- NEW: for browser download
import shutil, time, pandas as pd

#Drive root (same as before, to read your results)
ROOT = Path("/content/drive/MyDrive/AlpacaPaper")
TODAY = datetime.now(timezone.utc).strftime("%Y-%m-%d")

#Original sources in Drive (unchanged)
SRC_RESULTS = ROOT / "results" / TODAY         #e.g., /.../results/2025-10-13
SRC_EXPORT  = ROOT / "results_export" / TODAY  #rescue export folder (if used)

#=== CHANGE: write/export to LOCAL staging (in Colab VM), not Drive ===
DEST = Path("/content") / "exports" / f"{TODAY}_export"
DEST.mkdir(parents=True, exist_ok=True)

def copy_all(src_dir, dest_dir):
    if src_dir.exists():
        for p in src_dir.glob("*"):
            if p.is_file():
                shutil.copy2(p, dest_dir / p.name)
                print("Copied:", p.name, "from", src_dir.name)
    else:
        print("Missing source:", src_dir)

#Copy from both possible sources into local /content/exports/<today>_export
copy_all(SRC_RESULTS, DEST)
copy_all(SRC_EXPORT, DEST)

#Build/refresh trade_log_master.csv from per-symbol logs (in LOCAL DEST)
sym_logs = list(DEST.glob("trade_log_*.csv"))
if sym_logs:
    frames = []
    for p in sym_logs:
        try:
            df = pd.read_csv(p)
            df["symbol_file"] = p.stem.replace("trade_log_", "")
            frames.append(df)
        except Exception as e:
            print("Skip", p.name, "->", e)
    if frames:
        master = pd.concat(frames, ignore_index=True, sort=False)
        master_path = DEST / "trade_log_master.csv"
        master.to_csv(master_path, index=False)
        print("Wrote:", master_path)

#Zip LOCALLY under /content and trigger a browser download
zip_base = Path("/content") / f"results_{TODAY}_{int(time.time())}"
archive_path = shutil.make_archive(str(zip_base), "zip", DEST)
archive_path = str(Path(archive_path))  #ensure string for files.download

print("ZIP ->", archive_path)

#OPTIONAL: also keep a copy in Drive (uncomment if wanted)
#shutil.copy2(archive_path, ROOT / "results" / Path(archive_path).name)

#Prompt download to your computer
files.download(archive_path)

#Show what's in the LOCAL export folder
print("\nLocal export now contains:")
for p in sorted(DEST.iterdir()):
    print(" -", p.name)


Copied: live_loop.log from 2026-02-11
Copied: run_config.json from 2026-02-11
Copied: trade_log_master.csv from 2026-02-11
Copied: trade_log_UNH.csv from 2026-02-11
Copied: equity_log.csv from 2026-02-11
Copied: trade_log_GE.csv from 2026-02-11
Copied: equity_curve.png from 2026-02-11
Missing source: /content/drive/MyDrive/AlpacaPaper/results_export/2026-02-11
Wrote: /content/exports/2026-02-11_export/trade_log_master.csv
ZIP -> /content/results_2026-02-11_1770843610.zip


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Local export now contains:
 - equity_curve.png
 - equity_log.csv
 - live_loop.log
 - run_config.json
 - trade_log_GE.csv
 - trade_log_UNH.csv
 - trade_log_master.csv
